In [11]:
%reload_ext autoreload

%autoreload 2

In [ ]:
from minisom import MiniSom
from sklearn.preprocessing import LabelEncoder
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
import missingpy
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from GEMA import map
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from src import workflow as wf
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from matplotlib.patches import Patch
from matplotlib.colors import LinearSegmentedColormap

El error que se produce en la representación de GEMA es cosa de la librería pero no impide seguir ejecutando las cosas. TAmbién es posible ver como se ha representado osea que no influye en nada

## Importar dataset

In [3]:
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/oil-spill.csv'
missing_values = [' ', 'NaN', 'na', 'Na', '-', '--', 'n/a', '?']
columns = ['class']
data = pd.read_csv(url, na_values = missing_values, header=None)

In [ ]:
data

In [ ]:
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 49)
datos_mode[49].value_counts()

## Estandarización de los datos 

In [5]:
datos_mode=datos_mode.drop(columns = [49])
data_std = wf.min_max_normalization(datos_mode)
target = data[49].values
label_names={0:'Not_spiled', 1:'Spiled'}
data_som=data_std.to_numpy()

In [ ]:
print(data_som.shape[1])

In [ ]:
data_std

## Entrenamiento de SOM

In [ ]:
n_neurons = 16
m_neurons = 16

som = MiniSom(n_neurons, m_neurons, data_som.shape[1], learning_rate=0.24760602354016653, random_seed=42)
som.train(data_som, num_iteration=10000, verbose=True)  # random training
print("Topological error:",som.topographic_error(data_som))


In [ ]:
plt.figure(figsize=(n_neurons, m_neurons))

plt.pcolor(som.distance_map().T, cmap='Greys')  # plotting the distance map as background
plt.colorbar()

plt.show()

## Entrenamiento GEMA

In [ ]:
main_map = GEMA.Map(data_som, 
                    9, 
                    7500, 
                    initial_lr=0.20, 
                    initial_neighbourhood = 0,
                    distance = 'euclidean',
                    use_decay= False,
                    presentation= 'random',
                    weights = 'PCA')

main_classification = GEMA.Classification(main_map, data_som)
print(main_classification.quantization_error)
print(main_classification.topological_error)

In [ ]:
#GEMA.Visualization.codebook_vectors(main_map, np.array(["Not Spiled", "Spiled"]))

## SMOTE

In [ ]:
## Smote + Tomed_links
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 49)
data_std = wf.min_max_normalization(datos_mode)

Smote_tomed=wf.smote_tomed_link(data_std,49)
print(Smote_tomed[49].value_counts())
# Smote_tomed= Smote_tomed.reset_index(drop=True, inplace=True)
# data_std= data_std.reset_index(drop=True, inplace=True)
# diferentes=Smote_tomed[(Smote_tomed == data_std).all(axis=1) == False]
# print(diferentes)
data_balanced_dropped_smote_tomed = Smote_tomed.drop(columns = [49])
data_balanced_dropped_smote_tomed=data_balanced_dropped_smote_tomed.to_numpy()

## smote_edited_nearest_neighbor
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 49)
data_std = wf.min_max_normalization(datos_mode)

Smote_edited=wf.smote_edited_nearest_neighbor(data_std,49)
print(Smote_edited[49].value_counts())
data_balanced_dropped_smote_edited = Smote_edited.drop(columns = [49])
data_balanced_dropped_smote_edited=data_balanced_dropped_smote_edited.to_numpy()

## smote con CNN (condensed nearest neighbors)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 49)
data_std = wf.min_max_normalization(datos_mode)

Smote_cnn=wf.smote_cnn(data_std,49)
print(Smote_cnn[49].value_counts())
data_smote_cnn = Smote_cnn.drop(columns = [49])
data_smote_cnn=data_smote_cnn.to_numpy()

## smote con NCR (Neighboorhood cleaning rule)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 49)
data_std = wf.min_max_normalization(datos_mode)

Smote_ncr=wf.smote_ncr(data_std,49)
print(Smote_ncr[49].value_counts())
data_smote_ncr = Smote_ncr.drop(columns = [49])
data_smote_ncr=data_smote_ncr.to_numpy()

## smote con OSD (One sided selection)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 49)
data_std = wf.min_max_normalization(datos_mode)

Smote_oss=wf.smote_osd(data_std,49)
print(Smote_oss[49].value_counts())
data_smote_oss = Smote_oss.drop(columns = [49])
data_smote_oss=data_smote_oss.to_numpy()

## Clasificación con GEMA y SMOTE

In [ ]:
main_classification_smoted_tomed = GEMA.Classification(main_map, data_balanced_dropped_smote_tomed)
print("Smote+tommed quantification:",main_classification_smoted_tomed.quantization_error)
print("Smote+tommed topological:",main_classification_smoted_tomed.topological_error)

main_classification_smoted_edited= GEMA.Classification(main_map, data_balanced_dropped_smote_edited)
print("Smote+edited quantification:",main_classification_smoted_edited.quantization_error)
print("Smote+edited topological:",main_classification_smoted_edited.topological_error)

main_classification_smoted_cnn= GEMA.Classification(main_map, data_smote_cnn)
print("Smote+cnn quantification:",main_classification_smoted_cnn.quantization_error)
print("Smote+cnn topological:",main_classification_smoted_cnn.topological_error)

main_classification_smoted_ncr= GEMA.Classification(main_map, data_smote_ncr)
print("Smote+ncr quantification:",main_classification_smoted_ncr.quantization_error)
print("Smote+ncr topological:",main_classification_smoted_ncr.topological_error)

main_classification_smoted_oss= GEMA.Classification(main_map, data_smote_oss)
print("Smote+oss quantification:",main_classification_smoted_oss.quantization_error)
print("Smote+oss topological:",main_classification_smoted_oss.topological_error)

## Clasificacion con Minisom y SMOTE

In [ ]:
print("Smote+tommed")
print(som.quantization_error(data_balanced_dropped_smote_tomed), som.topographic_error(data_balanced_dropped_smote_tomed))
print("Smote+linked")
print(som.quantization_error(data_balanced_dropped_smote_edited), som.topographic_error(data_balanced_dropped_smote_edited))
print("Smote+cnn")
print(som.quantization_error(data_smote_cnn), som.topographic_error(data_smote_cnn))
print("Smote+ncr")
print(som.quantization_error(data_smote_ncr), som.topographic_error(data_smote_ncr))
print("Smote+oss")
print(som.quantization_error(data_smote_oss), som.topographic_error(data_smote_oss))

## ADASYN

In [ ]:
## ADASYN + Tomed_links
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 49)
data_std = wf.min_max_normalization(datos_mode)

adasyn_tomek=wf.adasyn_tomek(data_std,49)
print(adasyn_tomek[49].value_counts())
data_adasyn_tomek = adasyn_tomek.drop(columns = [49])
data_adasyn_tomek=data_adasyn_tomek.to_numpy()

## ADASYN + ENN
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 49)
data_std = wf.min_max_normalization(datos_mode)

adasyn_edited=wf.adasyn_enn(data_std,49)
print(adasyn_edited[49].value_counts())
data_adasyn_edited = adasyn_edited.drop(columns = [49])
data_adasyn_edited=data_adasyn_edited.to_numpy()

## ADASYN con CNN (condensed nearest neighbors)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 49)
data_std = wf.min_max_normalization(datos_mode)

adasyn_cnn=wf.adasyn_cnn(data_std,49)
print(adasyn_cnn[49].value_counts())
data_adasyn_cnn = adasyn_cnn.drop(columns = [49])
data_adasyn_cnn=data_adasyn_cnn.to_numpy()

## ADASYN con NCR (Neighboorhood cleaning rule)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 49)
data_std = wf.min_max_normalization(datos_mode)

adasyn_ncr=wf.adasyn_ncr(data_std,49)
print(adasyn_ncr[49].value_counts())
data_adasyn_ncr = adasyn_ncr.drop(columns = [49])
data_adasyn_ncr=data_adasyn_ncr.to_numpy()

## ADASYN con OSD (One sided selection)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 49)
data_std = wf.min_max_normalization(datos_mode)

adasyn_oss=wf.adasyn_oss(data_std,49)
print(adasyn_oss[49].value_counts())
data_adasyn_oss = adasyn_oss.drop(columns = [49])
data_adasyn_oss=data_adasyn_oss.to_numpy()

## Clasificación con GEMA y ADASYN

In [ ]:
main_classification_adasyn_tomek = GEMA.Classification(main_map, data_adasyn_tomek)
print("adasyn+tommed quantification:",main_classification_adasyn_tomek.quantization_error)
print("adasyn+tommed topological:",main_classification_adasyn_tomek.topological_error)

main_classification_adasyn_edited= GEMA.Classification(main_map, data_adasyn_edited)
print("adasyn+edited quantification:",main_classification_adasyn_edited.quantization_error)
print("adasyn+edited topological:",main_classification_adasyn_edited.topological_error)

main_classification_smoted_cnn= GEMA.Classification(main_map, data_smote_cnn)
print("adasyn+cnn quantification:",main_classification_smoted_cnn.quantization_error)
print("adasyn+cnn topological:",main_classification_smoted_cnn.topological_error)

main_classification_smoted_ncr= GEMA.Classification(main_map, data_smote_ncr)
print("adasyn+ncr quantification:",main_classification_smoted_ncr.quantization_error)
print("adasyn+ncr topological:",main_classification_smoted_ncr.topological_error)

main_classification_smoted_oss= GEMA.Classification(main_map, data_smote_oss)
print("adasyn+oss quantification:",main_classification_smoted_oss.quantization_error)
print("adasyn+oss topological:",main_classification_smoted_oss.topological_error)

## Clasificación con Minisom y ADASYN

In [ ]:
print("adasyn+tommed")
print(som.quantization_error(data_adasyn_tomek), som.topographic_error(data_adasyn_tomek))
print("adasyn+linked")
print(som.quantization_error(data_adasyn_edited), som.topographic_error(data_adasyn_edited))
print("adasyn+cnn")
print(som.quantization_error(data_adasyn_cnn), som.topographic_error(data_adasyn_cnn))
print("adasyn+ncr")
print(som.quantization_error(data_adasyn_ncr), som.topographic_error(data_adasyn_ncr))
print("adasyn+oss")
print(som.quantization_error(data_adasyn_oss), som.topographic_error(data_adasyn_oss))

## Borderline SMOTE

In [ ]:
## BSMOTE + Tomed_links
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 49)
data_std = wf.min_max_normalization(datos_mode)

bsmote_tomek=wf.BSMOTE_tomek(data_std,49)
print(bsmote_tomek[49].value_counts())
data_bsmote_tomek = bsmote_tomek.drop(columns = [49])
data_bsmote_tomek=data_bsmote_tomek.to_numpy()

## BSMOTE + ENN
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 49)
data_std = wf.min_max_normalization(datos_mode)

bsmote_edited=wf.BSMOTE_enn(data_std,49)
print(bsmote_edited[49].value_counts())
data_bsmote_edited = bsmote_edited.drop(columns = [49])
data_bsmote_edited=data_bsmote_edited.to_numpy()

## BSMOTE con CNN (condensed nearest neighbors)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 49)
data_std = wf.min_max_normalization(datos_mode)

bsmote_cnn=wf.BSMOTE_cnn(data_std,49)
print(bsmote_cnn[49].value_counts())
data_bsmote_cnn = bsmote_cnn.drop(columns = [49])
data_bsmote_cnn=data_bsmote_cnn.to_numpy()

## BSMOTE con NCR (Neighboorhood cleaning rule)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 49)
data_std = wf.min_max_normalization(datos_mode)

bsmote_ncr=wf.BSMOTE_ncr(data_std,49)
print(bsmote_ncr[49].value_counts())
data_bsmote_ncr = bsmote_ncr.drop(columns = [49])
data_bsmote_ncr=data_bsmote_ncr.to_numpy()

## BSMOTE con OSD (One sided selection)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 49)
data_std = wf.min_max_normalization(datos_mode)

bsmote_oss=wf.BSMOTE_oss(data_std,49)
print(bsmote_oss[49].value_counts())
data_bsmote_oss = bsmote_oss.drop(columns = [49])
data_bsmote_oss=data_bsmote_oss.to_numpy()

## Clasificación con GEMA y Borderline SMOTE

In [ ]:
main_classification_bsmote_tomek = GEMA.Classification(main_map, data_bsmote_tomek)
print("bsmote+tommed quantification:",main_classification_bsmote_tomek.quantization_error)
print("bsmote+tommed topological:",main_classification_bsmote_tomek.topological_error)

main_classification_bsmote_edited= GEMA.Classification(main_map, data_bsmote_edited)
print("bsmote+edited quantification:",main_classification_bsmote_edited.quantization_error)
print("bsmote+edited topological:",main_classification_bsmote_edited.topological_error)

main_classification_bsmote_cnn= GEMA.Classification(main_map, data_bsmote_cnn)
print("bsmote+cnn quantification:",main_classification_bsmote_cnn.quantization_error)
print("bsmote+cnn topological:",main_classification_bsmote_cnn.topological_error)

main_classification_bsmote_ncr= GEMA.Classification(main_map, data_bsmote_ncr)
print("bsmote+ncr quantification:",main_classification_bsmote_ncr.quantization_error)
print("bsmote+ncr topological:",main_classification_bsmote_ncr.topological_error)

main_classification_bsmote_oss= GEMA.Classification(main_map, data_bsmote_oss)
print("bsmote+oss quantification:",main_classification_bsmote_oss.quantization_error)
print("bsmote+oss topological:",main_classification_bsmote_oss.topological_error)

## Clasificación con Minisom y Borderline SMOTE

In [ ]:
print("bsmote+tommed")
print(som.quantization_error(data_bsmote_tomek), som.topographic_error(data_bsmote_tomek))
print("bsmote+linked")
print(som.quantization_error(data_bsmote_edited), som.topographic_error(data_bsmote_edited))
print("bsmote+cnn")
print(som.quantization_error(data_bsmote_cnn), som.topographic_error(data_bsmote_cnn))
print("bsmote+ncr")
print(som.quantization_error(data_bsmote_ncr), som.topographic_error(data_bsmote_ncr))
print("bsmote+oss")
print(som.quantization_error(data_bsmote_oss), som.topographic_error(data_bsmote_oss))

## SVM SMOTE

In [ ]:
## SVMSMOTE + Tomed_links
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 49)
data_std = wf.min_max_normalization(datos_mode)

svmsmote_tomek=wf.SVMSMOTE_tomek(data_std,49)
print(svmsmote_tomek[49].value_counts())
data_svmsmote_tomek = svmsmote_tomek.drop(columns = [49])
data_svmsmote_tomek=data_svmsmote_tomek.to_numpy()

## SVMSMOTE + ENN
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 49)
data_std = wf.min_max_normalization(datos_mode)

svmsmote_edited=wf.SVMSMOTE_enn(data_std,49)
print(svmsmote_edited[49].value_counts())
data_svmsmote_edited = svmsmote_edited.drop(columns = [49])
data_svmsmote_edited=data_svmsmote_edited.to_numpy()

## SVMSMOTE con CNN (condensed nearest neighbors)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 49)
data_std = wf.min_max_normalization(datos_mode)

svmsmote_cnn=wf.SVMSMOTE_cnn(data_std,49)
print(svmsmote_cnn[49].value_counts())
data_svmsmote_cnn = svmsmote_cnn.drop(columns = [49])
data_svmsmote_cnn=data_svmsmote_cnn.to_numpy()

## SVMSMOTE con NCR (Neighboorhood cleaning rule)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 49)
data_std = wf.min_max_normalization(datos_mode)

svmsmote_ncr=wf.SVMSMOTE_ncr(data_std,49)
print(svmsmote_ncr[49].value_counts())
data_svmsmote_ncr = svmsmote_ncr.drop(columns = [49])
data_svmsmote_ncr=data_svmsmote_ncr.to_numpy()

## SVMSMOTE con OSD (One sided selection)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 49)
data_std = wf.min_max_normalization(datos_mode)

svmsmote_oss=wf.SVMSMOTE_oss(data_std,49)
print(svmsmote_oss[49].value_counts())
data_svmsmote_oss = svmsmote_oss.drop(columns = [49])
data_svmsmote_oss=data_svmsmote_oss.to_numpy()

## Clasificación con GEMA y SVM SMOTE

In [ ]:
main_classification_svmsmote_tomek = GEMA.Classification(main_map, data_svmsmote_tomek)
print("svmsmote+tommed quantification:",main_classification_svmsmote_tomek.quantization_error)
print("svmsmote+tommed topological:",main_classification_svmsmote_tomek.topological_error)

main_classification_svmsmote_edited= GEMA.Classification(main_map, data_svmsmote_edited)
print("svmsmote+edited quantification:",main_classification_svmsmote_edited.quantization_error)
print("svmsmote+edited topological:",main_classification_svmsmote_edited.topological_error)

main_classification_svmsmote_cnn= GEMA.Classification(main_map, data_svmsmote_cnn)
print("svmsmote+cnn quantification:",main_classification_svmsmote_cnn.quantization_error)
print("svmsmote+cnn topological:",main_classification_svmsmote_cnn.topological_error)

main_classification_svmsmote_ncr= GEMA.Classification(main_map, data_svmsmote_ncr)
print("svmsmote+ncr quantification:",main_classification_svmsmote_ncr.quantization_error)
print("svmsmote+ncr topological:",main_classification_svmsmote_ncr.topological_error)

main_classification_svmsmote_oss= GEMA.Classification(main_map, data_svmsmote_oss)
print("svmsmote+oss quantification:",main_classification_svmsmote_oss.quantization_error)
print("svmsmote+oss topological:",main_classification_svmsmote_oss.topological_error)

## Clasificación con Minisom y SVM SMOTE

In [ ]:
print("svmsmote+tommed")
print(som.quantization_error(data_svmsmote_tomek), som.topographic_error(data_svmsmote_tomek))
print("svmsmote+linked")
print(som.quantization_error(data_svmsmote_edited), som.topographic_error(data_svmsmote_edited))
print("svmsmote+cnn")
print(som.quantization_error(data_svmsmote_cnn), som.topographic_error(data_svmsmote_cnn))
print("svmsmote+ncr")
print(som.quantization_error(data_svmsmote_ncr), som.topographic_error(data_svmsmote_ncr))
print("svmsmote+oss")
print(som.quantization_error(data_svmsmote_oss), som.topographic_error(data_svmsmote_oss))

## Kmeans SMOTE

In [ ]:
## KMSSMOTE + Tomed_links
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 49)
data_std = wf.min_max_normalization(datos_mode)

kmssmote_tomek=wf.KMSSMOTE_tomek(data_std,49)
print(kmssmote_tomek[49].value_counts())
data_kmssmote_tomek = kmssmote_tomek.drop(columns = [49])
data_kmssmote_tomek=data_kmssmote_tomek.to_numpy()

## KMSSMOTE + ENN
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 49)
data_std = wf.min_max_normalization(datos_mode)

kmssmote_edited=wf.KMSSMOTE_enn(data_std,49)
print(kmssmote_edited[49].value_counts())
data_kmssmote_edited = kmssmote_edited.drop(columns = [49])
data_kmssmote_edited=data_kmssmote_edited.to_numpy()

## KMSSMOTE con CNN (condensed nearest neighbors)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 49)
data_std = wf.min_max_normalization(datos_mode)

kmssmote_cnn=wf.KMSSMOTE_cnn(data_std,49)
print(kmssmote_cnn[49].value_counts())
data_kmssmote_cnn = kmssmote_cnn.drop(columns = [49])
data_kmssmote_cnn=data_kmssmote_cnn.to_numpy()

## KMSSMOTE con NCR (Neighboorhood cleaning rule)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 49)
data_std = wf.min_max_normalization(datos_mode)

kmssmote_ncr=wf.KMSSMOTE_ncr(data_std,49)
print(kmssmote_ncr[49].value_counts())
data_kmssmote_ncr = kmssmote_ncr.drop(columns = [49])
data_kmssmote_ncr=data_kmssmote_ncr.to_numpy()

## KMSSMOTE con OSD (One sided selection)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 49)
data_std = wf.min_max_normalization(datos_mode)

kmssmote_oss=wf.KMSSMOTE_oss(data_std,49)
print(kmssmote_oss[49].value_counts())
data_kmssmote_oss = kmssmote_oss.drop(columns = [49])
data_kmssmote_oss=data_kmssmote_oss.to_numpy()

## Clasificación con GEMA y Kmeans SMOTE

In [ ]:
main_classification_kmssmote_tomek = GEMA.Classification(main_map, data_kmssmote_tomek)
print("svmsmote+tommed quantification:",main_classification_kmssmote_tomek.quantization_error)
print("svmsmote+tommed topological:",main_classification_kmssmote_tomek.topological_error)

main_classification_kmssmote_edited= GEMA.Classification(main_map, data_kmssmote_edited)
print("svmsmote+edited quantification:",main_classification_kmssmote_edited.quantization_error)
print("svmsmote+edited topological:",main_classification_kmssmote_edited.topological_error)

main_classification_kmssmote_cnn= GEMA.Classification(main_map, data_kmssmote_cnn)
print("svmsmote+cnn quantification:",main_classification_kmssmote_cnn.quantization_error)
print("svmsmote+cnn topological:",main_classification_kmssmote_cnn.topological_error)

main_classification_kmssmote_ncr= GEMA.Classification(main_map, data_kmssmote_ncr)
print("svmsmote+ncr quantification:",main_classification_kmssmote_ncr.quantization_error)
print("svmsmote+ncr topological:",main_classification_kmssmote_ncr.topological_error)

main_classification_kmssmote_oss= GEMA.Classification(main_map, data_kmssmote_oss)
print("svmsmote+oss quantification:",main_classification_kmssmote_oss.quantization_error)
print("svmsmote+oss topological:",main_classification_kmssmote_oss.topological_error)

## Clasificación con Minisom y Kmeans SMOTE

In [ ]:
print("kmssmote+tommed")
print(som.quantization_error(data_kmssmote_tomek), som.topographic_error(data_kmssmote_tomek))
print("kmssmote+linked")
print(som.quantization_error(data_kmssmote_edited), som.topographic_error(data_kmssmote_edited))
print("kmssmote+cnn")
print(som.quantization_error(data_kmssmote_cnn), som.topographic_error(data_kmssmote_cnn))
print("kmssmote+ncr")
print(som.quantization_error(data_kmssmote_ncr), som.topographic_error(data_kmssmote_ncr))
print("kmssmote+oss")
print(som.quantization_error(data_kmssmote_oss), som.topographic_error(data_kmssmote_oss))

## Importar dataset Cancer de mama

In [2]:
## Importar dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/haberman.csv'
missing_values = [' ', 'NaN', 'na', 'Na', '-', '--', 'n/a', '?']
columns = ['class']
data = pd.read_csv(url, na_values = missing_values, header=None)
data[3] = data[3].replace({1: 0, 2: 1})

In [ ]:
data

In [ ]:
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 3)
datos_mode[3].value_counts()

In [4]:
datos_mode=datos_mode.drop(columns = [3])
data_std = wf.min_max_normalization(datos_mode)
target = data[3].values
label_names={0:'plus 5', 1:'minus 5'}
data_som=data_std.to_numpy()

In [ ]:
X = wf.min_max_normalization(data)
X = X.drop(X.columns[3], axis=1)
y = data.iloc[: , 3]

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

# Dividir el conjunto de entrenamiento en conjunto de entrenamiento y conjunto de validación
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, shuffle=True)

# Escalar las características para mejorar el rendimiento del modelo
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_validation = scaler.transform(X_validation)
X_test = scaler.transform(X_test)

# Crear un modelo secuencial en Keras
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=20))  # Capa de entrada con 20 características
model.add(Dense(units=32, activation='relu'))  # Capa oculta con 32 neuronas y función de activación ReLU
model.add(Dense(units=1, activation='sigmoid'))  # Capa de salida con una neurona y función de activación sigmoide

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_validation, y_validation))

## Entrenamiento GEMA cancer

In [ ]:
main_map = GEMA.Map(data_som, 
                    9, 
                    5000, 
                    initial_lr=0.3, 
                    initial_neighbourhood = 0,
                    distance = 'euclidean',
                    use_decay= False,
                    presentation= 'random',
                    weights = 'sample')

main_classification = GEMA.Classification(main_map, data_som)
print("quantification error:",main_classification.quantization_error)
print("Topological_error:",main_classification.topological_error)

In [ ]:
#import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.colors import LinearSegmentedColormap

porcentaje = 0.65

# Inicializa un mapa de colores del tamaño del SOM con todos los valores en blanco
color_map = np.ones((9, 9, 3))

# Inicializa un mapa de clases del tamaño del SOM con todos los valores en -1
class_map = -1 * np.ones((9, 9))

# Contadores para cada neurona
counts = np.zeros((9, 9, 2), dtype=int)

# Itera sobre cada dato y su clase correspondiente
for datum, class_ in zip(data_som, target):
    # Encuentra la neurona ganadora para este dato
    winner = main_map.calculate_bmu(datum)[1]
    
    # Actualiza el contador de la clase para la neurona ganadora
    counts[int(winner[0]), int(winner[1]), int(class_)] += 1
    
    # Actualiza el mapa de clases con la clase reconocida
    class_map[winner] = class_

# Itera sobre cada neurona para determinar el color final
for i in range(9):
    for j in range(9):
        # Calcula el porcentaje de cada clase para la neurona actual
        percent_class_0 = counts[i, j, 0] / (counts[i, j, 0] + counts[i, j, 1]) if counts[i, j, 0] + counts[i, j, 1] > 0 else 0
        percent_class_1 = counts[i, j, 1] / (counts[i, j, 0] + counts[i, j, 1]) if counts[i, j, 0] + counts[i, j, 1] > 0 else 0
        
        # Asigna el color según las condiciones especificadas
        if percent_class_0 >= porcentaje:
            color_map[i, j] = [0, 0, 1]  # Azul
        elif percent_class_1 >= porcentaje:
            color_map[i, j] = [1, 0, 0]  # Rojo
        elif counts[i, j, 0] + counts[i, j, 1] == 0:
            color_map[i, j] = [1, 1, 1]  # Blanco si no ha reconocido ningún patrón
        else:
            color_map[i, j] = [0, 1, 0]  # Verde si no alcanza el 80% de ninguna clase


# Count the number of neurons of each color
num_blue = np.sum(np.all(color_map == [0, 0, 1], axis=-1))
num_red = np.sum(np.all(color_map == [1, 0, 0], axis=-1))
num_white = np.sum(np.all(color_map == [1, 1, 1], axis=-1))
num_green = np.sum(np.all(color_map == [0, 1, 0], axis=-1))

# Crea una figura con diferentes tamaños para los subplots
fig = plt.figure(figsize=(8, 6))

# Crea una leyenda para el primer gráfico
legend_elements = [Patch(facecolor='blue', edgecolor='b', label='Clase 0'),
                   Patch(facecolor='red', edgecolor='r', label='Clase 1'),
                   Patch(facecolor='green', edgecolor='g', label='Ambas clases')]
ax1 = fig.add_subplot(121)  # 121 significa "1 fila, 2 columnas, primer gráfico"
# ax1.legend(handles=legend_elements, loc='upper right')
ax1.imshow(np.rot90(color_map))
ax1.set_title('Mapa de neuronas w/o escala')

# Inicializa un mapa de neuronas con ceros
neuron_map = np.zeros((9, 9))
ganadores = np.zeros((9, 9))

# Itera sobre los datos y las clases
for datum, clss in zip(data_som, target):
    # Obtiene la neurona ganadora para el dato actual
    winner = main_map.calculate_bmu(datum)[1]
    ganadores[winner] = 1
    print(winner, clss)
    # Si la clase es 0, resta 1, si la clase es 1, suma 1
    neuron_map[winner[0], winner[1]] += 1 if clss == 1 else -1

# Normaliza el mapa de neuronas para que los valores estén entre -1 y 1
# neuron_map = neuron_map / np.max(np.abs(neuron_map))

# Define los colores de la escala como azul, verde y amarillo
cmap_colors = ["indigo", "royalblue", "limegreen", "yellow"]

# Crea el mapa de colores
cmap = LinearSegmentedColormap.from_list("mycmap", cmap_colors)

# Dibuja el mapa de calor
ax2 = fig.add_subplot(122)  # 122 significa "1 fila, 2 columnas, segundo gráfico"
im = ax2.imshow(np.rot90(neuron_map), cmap=cmap, interpolation='nearest')
ax2.set_title('Mapa de neuronas con escala')

# Muestra la barra de colores
fig.colorbar(im, ax=ax2)

# Ajusta el layout
plt.tight_layout()

# Muestra el gráfico
plt.show()

# Print the number of neurons of each color
print(f"Number of blue neurons: {num_blue}")
print(f"Number of red neurons: {num_red}")
print(f"Number of white neurons: {num_white}")
print(f"Number of green neurons: {num_green}")

## Entrenamiento Minisom cancer

In [ ]:
n_neurons = 7
m_neurons = 7

som = MiniSom(n_neurons, m_neurons, data_som.shape[1], learning_rate=0.23097207947323742, random_seed=42)
som.train(data_som, num_iteration=5600, verbose=True)  # random training
print("Topological error:",som.topographic_error(data_som))

In [ ]:
u_matrix = visualization.Visualization.umatrix(main_classification)

In [ ]:
GEMA.Visualization.heat_map(main_classification)

In [ ]:
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

# Calcular la U-Matrix
u_matrix = som.distance_map()

# Visualizar la U-Matrix
plt.imshow(u_matrix, cmap='gray', origin='lower')
plt.colorbar()
plt.show()

# Asigna colores basados en la cuarta columna (índice 3)
data['color'] = ['blue' if x == 0 else 'orange' for x in data.iloc[:,3]]

fig = px.scatter_3d(data, x=data.columns[0], y=data.columns[1], z=data.columns[2], color='color')

fig.show()

## SMOTE Cancer

In [ ]:
## Smote + Tomed_links
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 3)
data_std = wf.min_max_normalization(datos_mode)

Smote_tomed=wf.smote_tomed_link(data_std,3)
print(Smote_tomed[3].value_counts())
# Smote_tomed= Smote_tomed.reset_index(drop=True, inplace=True)
# data_std= data_std.reset_index(drop=True, inplace=True)
# diferentes=Smote_tomed[(Smote_tomed == data_std).all(axis=1) == False]
# print(diferentes)
data_balanced_dropped_smote_tomed = Smote_tomed.drop(columns = [3])
data_balanced_dropped_smote_tomed=data_balanced_dropped_smote_tomed.to_numpy()

## smote_edited_nearest_neighbor
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 3)
data_std = wf.min_max_normalization(datos_mode)

Smote_edited=wf.smote_enn2(data_std,3)
print(Smote_edited[3].value_counts())
data_balanced_dropped_smote_edited = Smote_edited.drop(columns = [3])
data_balanced_dropped_smote_edited=data_balanced_dropped_smote_edited.to_numpy()

## smote con CNN (condensed nearest neighbors)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 3)
data_std = wf.min_max_normalization(datos_mode)

Smote_cnn=wf.smote_cnn(data_std,3)
print(Smote_cnn[3].value_counts())
data_smote_cnn = Smote_cnn.drop(columns = [3])
data_smote_cnn=data_smote_cnn.to_numpy()

## smote con NCR (Neighboorhood cleaning rule)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 3)
data_std = wf.min_max_normalization(datos_mode)

Smote_ncr=wf.smote_ncr(data_std,3)
print(Smote_ncr[3].value_counts())
data_smote_ncr = Smote_ncr.drop(columns = [3])
data_smote_ncr=data_smote_ncr.to_numpy()

## smote con OSD (One sided selection)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 3)
data_std = wf.min_max_normalization(datos_mode)

Smote_oss=wf.smote_osd(data_std,3)
print(Smote_oss[3].value_counts())
data_smote_oss = Smote_oss.drop(columns = [3])
data_smote_oss=data_smote_oss.to_numpy()

In [ ]:
y = Smote_tomed.iloc[: , 3]
X = Smote_tomed.drop(Smote_tomed.columns[3], axis=1)

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

# Dividir el conjunto de entrenamiento en conjunto de entrenamiento y conjunto de validación
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, shuffle=True)

# Escalar las características para mejorar el rendimiento del modelo
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valn = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Crear un modelo secuencial en Keras
model = Sequential()
model.add(Dense(units=45, activation='relu', input_dim=3))  # Capa de entrada con 20 características
model.add(Dense(units=45, activation='relu'))  # Capa oculta con 32 neuronas y función de activación ReLU
model.add(Dense(units=1, activation='sigmoid'))  # Capa de salida con una neurona y función de activación sigmoide
pesos = model.get_weights()
# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=25, batch_size=32, validation_data=(X_val, y_val))

# Calcular Accuracy en el conjunto de entrenamiento
train_accuracy = accuracy_score(y_train, model.predict(X_train) > 0.5)

# Calcular Accuracy en el conjunto de validación
validation_accuracy = accuracy_score(y_val, model.predict(X_val) > 0.5)

print(f'Training Accuracy: {train_accuracy}')
print(f'Validation Accuracy: {validation_accuracy}')

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)  # Convertir las probabilidades en etiquetas binarias (0 o 1)

# Calcular métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)

# Calcular la especificidad
TN = conf_matrix[0][0]  # Verdaderos negativos
FP = conf_matrix[0][1]  # Falsos positivos
specificity = TN / (TN + FP)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall (Sensibility): {recall}')
print(f'Specificity: {specificity}')

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)

# Crear la matriz de confusión gráfica
plt.figure(figsize=(6, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
y = Smote_ncr.iloc[: , 3]
X = Smote_ncr.drop(Smote_ncr.columns[3], axis=1)

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

# Dividir el conjunto de entrenamiento en conjunto de entrenamiento y conjunto de validación
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, shuffle=True)

# Escalar las características para mejorar el rendimiento del modelo
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valn = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Crear un modelo secuencial en Keras
model = Sequential()
model.add(Dense(units=45, activation='relu', input_dim=3))  # Capa de entrada con 20 características
model.add(Dense(units=45, activation='relu'))  # Capa oculta con 32 neuronas y función de activación ReLU
model.add(Dense(units=1, activation='sigmoid'))  # Capa de salida con una neurona y función de activación sigmoide
model.set_weights(pesos)
# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Calcular Accuracy en el conjunto de entrenamiento
train_accuracy = accuracy_score(y_train, model.predict(X_train) > 0.5)

# Calcular Accuracy en el conjunto de validación
validation_accuracy = accuracy_score(y_val, model.predict(X_val) > 0.5)

print(f'Training Accuracy: {train_accuracy}')
print(f'Validation Accuracy: {validation_accuracy}')

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)  # Convertir las probabilidades en etiquetas binarias (0 o 1)

# Calcular métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)

# Calcular la especificidad
TN = conf_matrix[0][0]  # Verdaderos negativos
FP = conf_matrix[0][1]  # Falsos positivos
specificity = TN / (TN + FP)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall (Sensibility): {recall}')
print(f'Specificity: {specificity}')

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)

# Crear la matriz de confusión gráfica
plt.figure(figsize=(6, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

## Clasificación cancer con GEMA y SMOTE

In [ ]:
main_classification_smoted_tomed = GEMA.Classification(main_map, data_balanced_dropped_smote_tomed)
print("Smote+tommed quantification:",main_classification_smoted_tomed.quantization_error)
print("Smote+tommed topological:",main_classification_smoted_tomed.topological_error)

main_classification_smoted_edited= GEMA.Classification(main_map, data_balanced_dropped_smote_edited)
print("Smote+edited quantification:",main_classification_smoted_edited.quantization_error)
print("Smote+edited topological:",main_classification_smoted_edited.topological_error)

main_classification_smoted_cnn= GEMA.Classification(main_map, data_smote_cnn)
print("Smote+cnn quantification:",main_classification_smoted_cnn.quantization_error)
print("Smote+cnn topological:",main_classification_smoted_cnn.topological_error)

main_classification_smoted_ncr= GEMA.Classification(main_map, data_smote_ncr)
print("Smote+ncr quantification:",main_classification_smoted_ncr.quantization_error)
print("Smote+ncr topological:",main_classification_smoted_ncr.topological_error)

main_classification_smoted_oss= GEMA.Classification(main_map, data_smote_oss)
print("Smote+oss quantification:",main_classification_smoted_oss.quantization_error)
print("Smote+oss topological:",main_classification_smoted_oss.topological_error)

In [ ]:
from imblearn.over_sampling import RandomOverSampler, SMOTE, BorderlineSMOTE, SVMSMOTE, ADASYN, KMeansSMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss, CondensedNearestNeighbour, TomekLinks, EditedNearestNeighbours, OneSidedSelection, NeighbourhoodCleaningRule
from imblearn.pipeline import Pipeline

In [ ]:
def smote_enn_clase0(dataset, target):

    print(dataset.head(5))
    print(dataset[3].value_counts())   

    # Separar las características y el target
    X = dataset.drop(target, axis=1)
    y = dataset[target]

    # Guardar los índices de las filas originales
    original_indices = X.index

    # Crear una instancia de SMOTE
    smote = SMOTE(sampling_strategy={0: int(len(y[y==0])*1.5), 1: len(y[y==1])})

    # Crear una instancia de Condensed Nearest Neighbour
    enn = EditedNearestNeighbours(sampling_strategy='all', n_neighbors= 1)
    pipeline = Pipeline([('SMOTE', smote), ('ENN', enn)])

    X_resampled, y_resampled = smote.fit_resample(X, y)

    # Redimensionamos las clases.
    y_resampled = y_resampled.to_numpy().reshape(-1, 1)
    y_resampled = y_resampled.reshape(np.size(X_resampled, 0),1)
    
    cols_df = dataset.columns
    cols_df = cols_df.drop(target)
    
    synthetic_data = pd.DataFrame(X_resampled, columns=cols_df)
    synthetic_data[target] = y_resampled

    # Eliminar las filas del conjunto de datos sobremuestreado que corresponden a las filas originales
    synthetic_data_only = synthetic_data.drop(original_indices)
                
    return synthetic_data_only

def smote_enn_clase1(dataset, target):

    print(dataset.head(5))
    print(dataset[3].value_counts())   

    # Separar las características y el target
    X = dataset.drop(target, axis=1)
    y = dataset[target]

    # Guardar los índices de las filas originales
    original_indices = X.index

    # Crear una instancia de SMOTE
    smote = SMOTE(sampling_strategy='auto')

    # Crear una instancia de Condensed Nearest Neighbour
    enn = EditedNearestNeighbours(sampling_strategy='auto', n_neighbors= 1)
    pipeline = Pipeline([('SMOTE', smote), ('ENN', enn)])

    X_resampled, y_resampled = pipeline.fit_resample(X, y)

    # Redimensionamos las clases.
    y_resampled = y_resampled.to_numpy().reshape(-1, 1)
    y_resampled = y_resampled.reshape(np.size(X_resampled, 0),1)
    
    cols_df = dataset.columns
    cols_df = cols_df.drop(target)
    
    synthetic_data = pd.DataFrame(X_resampled, columns=cols_df)
    synthetic_data[target] = y_resampled

    # Eliminar las filas del conjunto de datos sobremuestreado que corresponden a las filas originales
    synthetic_data_only = synthetic_data.drop(original_indices)
                
    return synthetic_data_only

Smote_enn_sinteticos0=smote_enn_clase0(data_std,3)
Smote_enn_sinteticos1=smote_enn_clase1(data_std,3)
smote_enn_sinteticos = pd.concat([Smote_enn_sinteticos0, Smote_enn_sinteticos1], axis=0, ignore_index=True)
print(smote_enn_sinteticos.head(5))
print(smote_enn_sinteticos[3].value_counts())


In [ ]:
y = adasyn_oss.iloc[: , 3]
X = adasyn_oss.drop(adasyn_oss.columns[3], axis=1)
y = y.to_numpy()
X = X.to_numpy()
porcentaje = 0.7

# Inicializa un mapa de colores del tamaño del SOM con todos los valores en blanco
color_map = np.ones((9, 9, 3))

# Inicializa un mapa de clases del tamaño del SOM con todos los valores en -1
class_map = -1 * np.ones((9, 9))

# Contadores para cada neurona
counts = np.zeros((9, 9, 2), dtype=int)

# Itera sobre cada dato y su clase correspondiente
for datum, class_ in zip(X, y):
    # Encuentra la neurona ganadora para este dato
    winner = main_map.calculate_bmu(datum)[1]
    
    # Actualiza el contador de la clase para la neurona ganadora
    counts[int(winner[0]), int(winner[1]), int(class_)] += 1
    
    # Actualiza el mapa de clases con la clase reconocida
    class_map[winner] = class_

# Itera sobre cada neurona para determinar el color final
for i in range(9):
    for j in range(9):
        # Calcula el porcentaje de cada clase para la neurona actual
        percent_class_0 = counts[i, j, 0] / (counts[i, j, 0] + counts[i, j, 1]) if counts[i, j, 0] + counts[i, j, 1] > 0 else 0
        percent_class_1 = counts[i, j, 1] / (counts[i, j, 0] + counts[i, j, 1]) if counts[i, j, 0] + counts[i, j, 1] > 0 else 0
        
        # Asigna el color según las condiciones especificadas
        if percent_class_0 >= porcentaje:
            color_map[i, j] = [0, 0, 1]  # Azul
        elif percent_class_1 >= porcentaje:
            color_map[i, j] = [1, 0, 0]  # Rojo
        elif counts[i, j, 0] + counts[i, j, 1] == 0:
            color_map[i, j] = [1, 1, 1]  # Blanco si no ha reconocido ningún patrón
        else:
            color_map[i, j] = [0, 1, 0]  # Verde si no alcanza el % de ninguna clase

# Crea una figura con diferentes tamaños para los subplots
fig = plt.figure(figsize=(8, 6))

# Crea una leyenda para el primer gráfico
legend_elements = [Patch(facecolor='blue', edgecolor='b', label='Clase 0'),
                   Patch(facecolor='red', edgecolor='r', label='Clase 1'),
                   Patch(facecolor='green', edgecolor='g', label='Ambas clases')]
ax1 = fig.add_subplot(121)  # 121 significa "1 fila, 2 columnas, primer gráfico"
#ax1.legend(handles=legend_elements, loc='upper right')
ax1.imshow(np.rot90(color_map))
ax1.set_title('Mapa de neuronas w/o escala')

# Inicializa un mapa de neuronas con ceros
neuron_map = np.zeros((9, 9))
ganadores = np.zeros((9, 9))

# Itera sobre los datos y las clases
for datum, clss in zip(X, y):
    # Obtiene la neurona ganadora para el dato actual
    winner = main_map.calculate_bmu(datum)[1]
    ganadores[winner] = 1
    print(winner, clss)
    # Si la clase es 0, resta 1, si la clase es 1, suma 1
    neuron_map[winner[0], winner[1]] += 1 if clss == 1 else -1

# Normaliza el mapa de neuronas para que los valores estén entre -1 y 1
# neuron_map = neuron_map / np.max(np.abs(neuron_map))

# Define los colores de la escala como azul, verde y amarillo
cmap_colors = ["indigo", "royalblue", "limegreen", "yellow"]

# Crea el mapa de colores
cmap = LinearSegmentedColormap.from_list("mycmap", cmap_colors)

# Dibuja el mapa de calor
ax2 = fig.add_subplot(122)  # 122 significa "1 fila, 2 columnas, segundo gráfico"
im = ax2.imshow(np.rot90(neuron_map), cmap=cmap, interpolation='nearest')
ax2.set_title('Mapa de neuronas con escala')

# Muestra la barra de colores
fig.colorbar(im, ax=ax2)

# Ajusta el layout
plt.tight_layout()

# Muestra el gráfico
plt.show()

num_blue = np.sum(np.all(color_map == [0, 0, 1], axis=-1))
num_red = np.sum(np.all(color_map == [1, 0, 0], axis=-1))
num_white = np.sum(np.all(color_map == [1, 1, 1], axis=-1))
num_green = np.sum(np.all(color_map == [0, 1, 0], axis=-1))
# Print the number of neurons of each color
print(f"Number of blue neurons: {num_blue}")
print(f"Number of red neurons: {num_red}")
print(f"Number of white neurons: {num_white}")
print(f"Number of green neurons: {num_green}")

data_balanced_dropped_smote_edited_sint = smote_enn_sinteticos.drop(columns = [3])
data_balanced_dropped_smote_edited_sint=data_balanced_dropped_smote_edited_sint.to_numpy()
main_classification_smoted_enn_sinteticos = GEMA.Classification(main_map, data_balanced_dropped_smote_edited_sint)
GEMA.Visualization.heat_map(main_classification_smoted_enn_sinteticos)

## Clasificación cancer con Minisom y SMOTE

In [ ]:
print("Smote+tommed")
print(som.quantization_error(data_balanced_dropped_smote_tomed), som.topographic_error(data_balanced_dropped_smote_tomed))
print("Smote+linked")
print(som.quantization_error(data_balanced_dropped_smote_edited), som.topographic_error(data_balanced_dropped_smote_edited))
print("Smote+cnn")
print(som.quantization_error(data_smote_cnn), som.topographic_error(data_smote_cnn))
print("Smote+ncr")
print(som.quantization_error(data_smote_ncr), som.topographic_error(data_smote_ncr))
print("Smote+oss")
print(som.quantization_error(data_smote_oss), som.topographic_error(data_smote_oss))

## ADASYN Cancer

In [ ]:
## ADASYN + Tomed_links
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 3)
data_std = wf.min_max_normalization(datos_mode)

adasyn_tomek=wf.adasyn_tomek(data_std,3)
print(adasyn_tomek[3].value_counts())
data_adasyn_tomek = adasyn_tomek.drop(columns = [3])
data_adasyn_tomek=data_adasyn_tomek.to_numpy()

## ADASYN + ENN
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 3)
data_std = wf.min_max_normalization(datos_mode)

adasyn_edited=wf.adasyn_enn(data_std,3)
print(adasyn_edited[3].value_counts())
data_adasyn_edited = adasyn_edited.drop(columns = [3])
data_adasyn_edited=data_adasyn_edited.to_numpy()

## ADASYN con CNN (condensed nearest neighbors)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 3)
data_std = wf.min_max_normalization(datos_mode)

adasyn_cnn=wf.adasyn_cnn(data_std,3)
print(adasyn_cnn[3].value_counts())
data_adasyn_cnn = adasyn_cnn.drop(columns = [3])
data_adasyn_cnn=data_adasyn_cnn.to_numpy()

## ADASYN con NCR (Neighboorhood cleaning rule)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 3)
data_std = wf.min_max_normalization(datos_mode)

adasyn_ncr=wf.adasyn_ncr(data_std,3)
print(adasyn_ncr[3].value_counts())
data_adasyn_ncr = adasyn_ncr.drop(columns = [3])
data_adasyn_ncr=data_adasyn_ncr.to_numpy()

## ADASYN con OSD (One sided selection)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 3)
data_std = wf.min_max_normalization(datos_mode)

adasyn_oss=wf.adasyn_oss(data_std,3)
print(adasyn_oss[3].value_counts())
data_adasyn_oss = adasyn_oss.drop(columns = [3])
data_adasyn_oss=data_adasyn_oss.to_numpy()

In [ ]:
y = adasyn_tomek.iloc[: , 3]
X = adasyn_tomek.drop(adasyn_tomek.columns[3], axis=1)

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

# Dividir el conjunto de entrenamiento en conjunto de entrenamiento y conjunto de validación
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, shuffle=True)

# Escalar las características para mejorar el rendimiento del modelo
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valn = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Crear un modelo secuencial en Keras
model = Sequential()
model.add(Dense(units=45, activation='relu', input_dim=3))  # Capa de entrada con 20 características
model.add(Dense(units=45, activation='relu'))  # Capa oculta con 32 neuronas y función de activación ReLU
model.add(Dense(units=1, activation='sigmoid'))  # Capa de salida con una neurona y función de activación sigmoide
pesos = model.get_weights()
# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Calcular Accuracy en el conjunto de entrenamiento
train_accuracy = accuracy_score(y_train, model.predict(X_train) > 0.5)

# Calcular Accuracy en el conjunto de validación
validation_accuracy = accuracy_score(y_val, model.predict(X_val) > 0.5)

print(f'Training Accuracy: {train_accuracy}')
print(f'Validation Accuracy: {validation_accuracy}')

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)  # Convertir las probabilidades en etiquetas binarias (0 o 1)

# Calcular métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)

# Calcular la especificidad
TN = conf_matrix[0][0]  # Verdaderos negativos
FP = conf_matrix[0][1]  # Falsos positivos
specificity = TN / (TN + FP)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall (Sensibility): {recall}')
print(f'Specificity: {specificity}')

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)

# Crear la matriz de confusión gráfica
plt.figure(figsize=(6, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
y = adasyn_oss.iloc[: , 3]
X = adasyn_oss.drop(adasyn_oss.columns[3], axis=1)

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

# Dividir el conjunto de entrenamiento en conjunto de entrenamiento y conjunto de validación
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, shuffle=True)

# Escalar las características para mejorar el rendimiento del modelo
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valn = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Crear un modelo secuencial en Keras
model = Sequential()
model.add(Dense(units=45, activation='relu', input_dim=3))  # Capa de entrada con 20 características
model.add(Dense(units=45, activation='relu'))  # Capa oculta con 32 neuronas y función de activación ReLU
model.add(Dense(units=1, activation='sigmoid'))  # Capa de salida con una neurona y función de activación sigmoide
model.set_weights(pesos)
# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Calcular Accuracy en el conjunto de entrenamiento
train_accuracy = accuracy_score(y_train, model.predict(X_train) > 0.5)

# Calcular Accuracy en el conjunto de validación
validation_accuracy = accuracy_score(y_val, model.predict(X_val) > 0.5)

print(f'Training Accuracy: {train_accuracy}')
print(f'Validation Accuracy: {validation_accuracy}')

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)  # Convertir las probabilidades en etiquetas binarias (0 o 1)

# Calcular métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)

# Calcular la especificidad
TN = conf_matrix[0][0]  # Verdaderos negativos
FP = conf_matrix[0][1]  # Falsos positivos
specificity = TN / (TN + FP)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall (Sensibility): {recall}')
print(f'Specificity: {specificity}')

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)

# Crear la matriz de confusión gráfica
plt.figure(figsize=(6, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

## Clasificación cancer con GEMA y ADASYN

In [ ]:
main_classification_adasyn_tomek = GEMA.Classification(main_map, data_adasyn_tomek)
print("adasyn+tommed quantification:",main_classification_adasyn_tomek.quantization_error)
print("adasyn+tommed topological:",main_classification_adasyn_tomek.topological_error)

main_classification_adasyn_edited= GEMA.Classification(main_map, data_adasyn_edited)
print("adasyn+edited quantification:",main_classification_adasyn_edited.quantization_error)
print("adasyn+edited topological:",main_classification_adasyn_edited.topological_error)

main_classification_adasyn_cnn= GEMA.Classification(main_map, data_adasyn_cnn)
print("adasyn+cnn quantification:",main_classification_adasyn_cnn.quantization_error)
print("adasyn+cnn topological:",main_classification_adasyn_cnn.topological_error)

main_classification_adasyn_ncr= GEMA.Classification(main_map, data_adasyn_ncr)
print("adasyn+ncr quantification:",main_classification_adasyn_ncr.quantization_error)
print("adasyn+ncr topological:",main_classification_adasyn_ncr.topological_error)

main_classification_adasyn_oss= GEMA.Classification(main_map, data_adasyn_oss)
print("adasyn+oss quantification:",main_classification_adasyn_oss.quantization_error)
print("adasyn+oss topological:",main_classification_adasyn_oss.topological_error)

In [ ]:
def smote_enn_clase0(dataset, target):

    print(dataset.head(5))
    print(dataset[3].value_counts())   

    # Separar las características y el target
    X = dataset.drop(target, axis=1)
    y = dataset[target]

    # Guardar los índices de las filas originales
    original_indices = X.index

    # Crear una instancia de SMOTE
    smote = ADASYN(sampling_strategy={0: int(len(y[y==0])*1.6), 1: len(y[y==1])})

    # Crear una instancia de Condensed Nearest Neighbour
    enn = NeighbourhoodCleaningRule(sampling_strategy='all', n_neighbors= 1)
    pipeline = Pipeline([('SMOTE', smote), ('ENN', enn)])

    X_resampled, y_resampled = smote.fit_resample(X, y)

    # Redimensionamos las clases.
    y_resampled = y_resampled.to_numpy().reshape(-1, 1)
    y_resampled = y_resampled.reshape(np.size(X_resampled, 0),1)
    
    cols_df = dataset.columns
    cols_df = cols_df.drop(target)
    
    synthetic_data = pd.DataFrame(X_resampled, columns=cols_df)
    synthetic_data[target] = y_resampled

    # Eliminar las filas del conjunto de datos sobremuestreado que corresponden a las filas originales
    synthetic_data_only = synthetic_data.drop(original_indices)
                
    return synthetic_data_only

def smote_enn_clase1(dataset, target):

    print(dataset.head(5))
    print(dataset[3].value_counts())   

    # Separar las características y el target
    X = dataset.drop(target, axis=1)
    y = dataset[target]

    # Guardar los índices de las filas originales
    original_indices = X.index

    # Crear una instancia de SMOTE
    smote = ADASYN(sampling_strategy='auto')

    # Crear una instancia de Condensed Nearest Neighbour
    enn = NeighbourhoodCleaningRule(sampling_strategy='auto', n_neighbors= 1)
    pipeline = Pipeline([('SMOTE', smote), ('ENN', enn)])

    X_resampled, y_resampled = pipeline.fit_resample(X, y)

    # Redimensionamos las clases.
    y_resampled = y_resampled.to_numpy().reshape(-1, 1)
    y_resampled = y_resampled.reshape(np.size(X_resampled, 0),1)
    
    cols_df = dataset.columns
    cols_df = cols_df.drop(target)
    
    synthetic_data = pd.DataFrame(X_resampled, columns=cols_df)
    synthetic_data[target] = y_resampled

    # Eliminar las filas del conjunto de datos sobremuestreado que corresponden a las filas originales
    synthetic_data_only = synthetic_data.drop(original_indices)
                
    return synthetic_data_only

Smote_enn_sinteticos0=smote_enn_clase0(data_std,3)
Smote_enn_sinteticos1=smote_enn_clase1(data_std,3)
smote_enn_sinteticos = pd.concat([Smote_enn_sinteticos0, Smote_enn_sinteticos1], axis=0, ignore_index=True)
print(smote_enn_sinteticos.head(5))
print(smote_enn_sinteticos[3].value_counts())


In [ ]:
y = smote_enn_sinteticos.iloc[: , 3]
X = smote_enn_sinteticos.drop(smote_enn_sinteticos.columns[3], axis=1)
y = y.to_numpy()
X = X.to_numpy()
porcentaje = 0.8

# Inicializa un mapa de colores del tamaño del SOM con todos los valores en blanco
color_map = np.ones((9, 9, 3))

# Inicializa un mapa de clases del tamaño del SOM con todos los valores en -1
class_map = -1 * np.ones((9, 9))

# Contadores para cada neurona
counts = np.zeros((9, 9, 2), dtype=int)

# Itera sobre cada dato y su clase correspondiente
for datum, class_ in zip(X, y):
    # Encuentra la neurona ganadora para este dato
    winner = main_map.calculate_bmu(datum)[1]
    
    # Actualiza el contador de la clase para la neurona ganadora
    counts[int(winner[0]), int(winner[1]), int(class_)] += 1
    
    # Actualiza el mapa de clases con la clase reconocida
    class_map[winner] = class_

# Itera sobre cada neurona para determinar el color final
for i in range(9):
    for j in range(9):
        # Calcula el porcentaje de cada clase para la neurona actual
        percent_class_0 = counts[i, j, 0] / (counts[i, j, 0] + counts[i, j, 1]) if counts[i, j, 0] + counts[i, j, 1] > 0 else 0
        percent_class_1 = counts[i, j, 1] / (counts[i, j, 0] + counts[i, j, 1]) if counts[i, j, 0] + counts[i, j, 1] > 0 else 0
        
        # Asigna el color según las condiciones especificadas
        if percent_class_0 >= porcentaje:
            color_map[i, j] = [0, 0, 1]  # Azul
        elif percent_class_1 >= porcentaje:
            color_map[i, j] = [1, 0, 0]  # Rojo
        elif counts[i, j, 0] + counts[i, j, 1] == 0:
            color_map[i, j] = [1, 1, 1]  # Blanco si no ha reconocido ningún patrón
        else:
            color_map[i, j] = [0, 1, 0]  # Verde si no alcanza el 80% de ninguna clase

# Crea una figura con diferentes tamaños para los subplots
fig = plt.figure(figsize=(8, 6))

# Crea una leyenda para el primer gráfico
legend_elements = [Patch(facecolor='blue', edgecolor='b', label='Clase 0'),
                   Patch(facecolor='red', edgecolor='r', label='Clase 1'),
                   Patch(facecolor='green', edgecolor='g', label='Ambas clases')]
ax1 = fig.add_subplot(121)  # 121 significa "1 fila, 2 columnas, primer gráfico"
#ax1.legend(handles=legend_elements, loc='upper right')
ax1.imshow(np.rot90(color_map))
ax1.set_title('Mapa de neuronas w/o escala')

# Inicializa un mapa de neuronas con ceros
neuron_map = np.zeros((9, 9))
ganadores = np.zeros((9, 9))

# Itera sobre los datos y las clases
for datum, clss in zip(X, y):
    # Obtiene la neurona ganadora para el dato actual
    winner = main_map.calculate_bmu(datum)[1]
    ganadores[winner] = 1
    print(winner, clss)
    # Si la clase es 0, resta 1, si la clase es 1, suma 1
    neuron_map[winner[0], winner[1]] += 1 if clss == 1 else -1

# Normaliza el mapa de neuronas para que los valores estén entre -1 y 1
# neuron_map = neuron_map / np.max(np.abs(neuron_map))

# Define los colores de la escala como azul, verde y amarillo
cmap_colors = ["indigo", "royalblue", "limegreen", "yellow"]

# Crea el mapa de colores
cmap = LinearSegmentedColormap.from_list("mycmap", cmap_colors)

# Dibuja el mapa de calor
ax2 = fig.add_subplot(122)  # 122 significa "1 fila, 2 columnas, segundo gráfico"
im = ax2.imshow(np.rot90(neuron_map), cmap=cmap, interpolation='nearest')
ax2.set_title('Mapa de neuronas con escala')

# Muestra la barra de colores
fig.colorbar(im, ax=ax2)

# Ajusta el layout
plt.tight_layout()

# Muestra el gráfico
plt.show()

data_balanced_dropped_smote_edited_sint = smote_enn_sinteticos.drop(columns = [3])
data_balanced_dropped_smote_edited_sint=data_balanced_dropped_smote_edited_sint.to_numpy()
main_classification_smoted_enn_sinteticos = GEMA.Classification(main_map, data_balanced_dropped_smote_edited_sint)
GEMA.Visualization.heat_map(main_classification_smoted_enn_sinteticos)

## Clasificación cancer con Minisom y ADASYN

In [ ]:
print("adasyn+tommed")
print(som.quantization_error(data_adasyn_tomek), som.topographic_error(data_adasyn_tomek))
print("adasyn+linked")
print(som.quantization_error(data_adasyn_edited), som.topographic_error(data_adasyn_edited))
print("adasyn+cnn")
print(som.quantization_error(data_adasyn_cnn), som.topographic_error(data_adasyn_cnn))
print("adasyn+ncr")
print(som.quantization_error(data_adasyn_ncr), som.topographic_error(data_adasyn_ncr))
print("adasyn+oss")
print(som.quantization_error(data_adasyn_oss), som.topographic_error(data_adasyn_oss))

## Borderline SMOTE Cancer

In [ ]:
## BSMOTE + Tomed_links
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 3)
data_std = wf.min_max_normalization(datos_mode)

bsmote_tomek=wf.BSMOTE_tomek(data_std,3)
print(bsmote_tomek[3].value_counts())
data_bsmote_tomek = bsmote_tomek.drop(columns = [3])
data_bsmote_tomek=data_bsmote_tomek.to_numpy()

## BSMOTE + ENN
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 3)
data_std = wf.min_max_normalization(datos_mode)

bsmote_edited=wf.BSMOTE_enn(data_std,3)
print(bsmote_edited[3].value_counts())
data_bsmote_edited = bsmote_edited.drop(columns = [3])
data_bsmote_edited=data_bsmote_edited.to_numpy()

## BSMOTE con CNN (condensed nearest neighbors)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 3)
data_std = wf.min_max_normalization(datos_mode)

bsmote_cnn=wf.BSMOTE_cnn(data_std,3)
print(bsmote_cnn[3].value_counts())
data_bsmote_cnn = bsmote_cnn.drop(columns = [3])
data_bsmote_cnn=data_bsmote_cnn.to_numpy()

## BSMOTE con NCR (Neighboorhood cleaning rule)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 3)
data_std = wf.min_max_normalization(datos_mode)

bsmote_ncr=wf.BSMOTE_ncr(data_std,3)
print(bsmote_ncr[3].value_counts())
data_bsmote_ncr = bsmote_ncr.drop(columns = [3])
data_bsmote_ncr=data_bsmote_ncr.to_numpy()

## BSMOTE con OSD (One sided selection)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 3)
data_std = wf.min_max_normalization(datos_mode)

bsmote_oss=wf.BSMOTE_oss(data_std,3)
print(bsmote_oss[3].value_counts())
data_bsmote_oss = bsmote_oss.drop(columns = [3])
data_bsmote_oss=data_bsmote_oss.to_numpy()

In [ ]:
y = kmssmote_oss.iloc[: , 3]
X = kmssmote_oss.drop(kmssmote_oss.columns[3], axis=1)
y = y.to_numpy()
X = X.to_numpy()
porcentaje = 0.8

# Inicializa un mapa de colores del tamaño del SOM con todos los valores en blanco
color_map = np.ones((9, 9, 3))

# Inicializa un mapa de clases del tamaño del SOM con todos los valores en -1
class_map = -1 * np.ones((9, 9))

# Contadores para cada neurona
counts = np.zeros((9, 9, 2), dtype=int)

# Itera sobre cada dato y su clase correspondiente
for datum, class_ in zip(X, y):
    # Encuentra la neurona ganadora para este dato
    winner = main_map.calculate_bmu(datum)[1]
    
    # Actualiza el contador de la clase para la neurona ganadora
    counts[int(winner[0]), int(winner[1]), int(class_)] += 1
    
    # Actualiza el mapa de clases con la clase reconocida
    class_map[winner] = class_

# Itera sobre cada neurona para determinar el color final
for i in range(9):
    for j in range(9):
        # Calcula el porcentaje de cada clase para la neurona actual
        percent_class_0 = counts[i, j, 0] / (counts[i, j, 0] + counts[i, j, 1]) if counts[i, j, 0] + counts[i, j, 1] > 0 else 0
        percent_class_1 = counts[i, j, 1] / (counts[i, j, 0] + counts[i, j, 1]) if counts[i, j, 0] + counts[i, j, 1] > 0 else 0
        
        # Asigna el color según las condiciones especificadas
        if percent_class_0 >= porcentaje:
            color_map[i, j] = [0, 0, 1]  # Azul
        elif percent_class_1 >= porcentaje:
            color_map[i, j] = [1, 0, 0]  # Rojo
        elif counts[i, j, 0] + counts[i, j, 1] == 0:
            color_map[i, j] = [1, 1, 1]  # Blanco si no ha reconocido ningún patrón
        else:
            color_map[i, j] = [0, 1, 0]  # Verde si no alcanza el % de ninguna clase

# Crea una figura con diferentes tamaños para los subplots
fig = plt.figure(figsize=(8, 6))

# Crea una leyenda para el primer gráfico
legend_elements = [Patch(facecolor='blue', edgecolor='b', label='Clase 0'),
                   Patch(facecolor='red', edgecolor='r', label='Clase 1'),
                   Patch(facecolor='green', edgecolor='g', label='Ambas clases')]
ax1 = fig.add_subplot(121)  # 121 significa "1 fila, 2 columnas, primer gráfico"
#ax1.legend(handles=legend_elements, loc='upper right')
ax1.imshow(np.rot90(color_map))
ax1.set_title('Mapa de neuronas w/o escala')

# Inicializa un mapa de neuronas con ceros
neuron_map = np.zeros((9, 9))
ganadores = np.zeros((9, 9))

# Itera sobre los datos y las clases
for datum, clss in zip(X, y):
    # Obtiene la neurona ganadora para el dato actual
    winner = main_map.calculate_bmu(datum)[1]
    ganadores[winner] = 1
    # Si la clase es 0, resta 1, si la clase es 1, suma 1
    neuron_map[winner[0], winner[1]] += 1 if clss == 1 else -1

# Normaliza el mapa de neuronas para que los valores estén entre -1 y 1
# neuron_map = neuron_map / np.max(np.abs(neuron_map))

# Define los colores de la escala como azul, verde y amarillo
cmap_colors = ["indigo", "royalblue", "limegreen", "yellow"]

# Crea el mapa de colores
cmap = LinearSegmentedColormap.from_list("mycmap", cmap_colors)

# Dibuja el mapa de calor
ax2 = fig.add_subplot(122)  # 122 significa "1 fila, 2 columnas, segundo gráfico"
im = ax2.imshow(np.rot90(neuron_map), cmap=cmap, interpolation='nearest')
ax2.set_title('Mapa de neuronas con escala')

# Muestra la barra de colores
fig.colorbar(im, ax=ax2)

# Ajusta el layout
plt.tight_layout()

# Muestra el gráfico
plt.show()

num_blue = np.sum(np.all(color_map == [0, 0, 1], axis=-1))
num_red = np.sum(np.all(color_map == [1, 0, 0], axis=-1))
num_white = np.sum(np.all(color_map == [1, 1, 1], axis=-1))
num_green = np.sum(np.all(color_map == [0, 1, 0], axis=-1))
# Print the number of neurons of each color
print(f"Number of blue neurons: {num_blue}")
print(f"Number of red neurons: {num_red}")
print(f"Number of white neurons: {num_white}")
print(f"Number of green neurons: {num_green}")

data_balanced_dropped_smote_edited_sint = smote_enn_sinteticos.drop(columns = [3])
data_balanced_dropped_smote_edited_sint=data_balanced_dropped_smote_edited_sint.to_numpy()
main_classification_smoted_enn_sinteticos = GEMA.Classification(main_map, data_balanced_dropped_smote_edited_sint)
GEMA.Visualization.heat_map(main_classification_smoted_enn_sinteticos)

In [ ]:
y = bsmote_tomek.iloc[: , 3]
X = bsmote_tomek.drop(bsmote_tomek.columns[3], axis=1)

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

# Dividir el conjunto de entrenamiento en conjunto de entrenamiento y conjunto de validación
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, shuffle=True)

# Escalar las características para mejorar el rendimiento del modelo
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valn = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Crear un modelo secuencial en Keras
model = Sequential()
model.add(Dense(units=45, activation='relu', input_dim=3))  # Capa de entrada con 20 características
model.add(Dense(units=45, activation='relu'))  # Capa oculta con 32 neuronas y función de activación ReLU
model.add(Dense(units=1, activation='sigmoid'))  # Capa de salida con una neurona y función de activación sigmoide
pesos = model.get_weights()
# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Calcular Accuracy en el conjunto de entrenamiento
train_accuracy = accuracy_score(y_train, model.predict(X_train) > 0.5)

# Calcular Accuracy en el conjunto de validación
validation_accuracy = accuracy_score(y_val, model.predict(X_val) > 0.5)

print(f'Training Accuracy: {train_accuracy}')
print(f'Validation Accuracy: {validation_accuracy}')

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)  # Convertir las probabilidades en etiquetas binarias (0 o 1)

# Calcular métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)

# Calcular la especificidad
TN = conf_matrix[0][0]  # Verdaderos negativos
FP = conf_matrix[0][1]  # Falsos positivos
specificity = TN / (TN + FP)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall (Sensibility): {recall}')
print(f'Specificity: {specificity}')

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)

# Crear la matriz de confusión gráfica
plt.figure(figsize=(6, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


In [ ]:
y = bsmote_oss.iloc[: , 3]
X = bsmote_oss.drop(bsmote_oss.columns[3], axis=1)

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

# Dividir el conjunto de entrenamiento en conjunto de entrenamiento y conjunto de validación
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, shuffle=True)

# Escalar las características para mejorar el rendimiento del modelo
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valn = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Crear un modelo secuencial en Keras
model = Sequential()
model.add(Dense(units=45, activation='relu', input_dim=3))  # Capa de entrada con 20 características
model.add(Dense(units=45, activation='relu'))  # Capa oculta con 32 neuronas y función de activación ReLU
model.add(Dense(units=1, activation='sigmoid'))  # Capa de salida con una neurona y función de activación sigmoide
model.set_weights(pesos)
# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=25, batch_size=32, validation_data=(X_val, y_val))

# Calcular Accuracy en el conjunto de entrenamiento
train_accuracy = accuracy_score(y_train, model.predict(X_train) > 0.5)

# Calcular Accuracy en el conjunto de validación
validation_accuracy = accuracy_score(y_val, model.predict(X_val) > 0.5)

print(f'Training Accuracy: {train_accuracy}')
print(f'Validation Accuracy: {validation_accuracy}')

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)  # Convertir las probabilidades en etiquetas binarias (0 o 1)

# Calcular métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)

# Calcular la especificidad
TN = conf_matrix[0][0]  # Verdaderos negativos
FP = conf_matrix[0][1]  # Falsos positivos
specificity = TN / (TN + FP)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall (Sensibility): {recall}')
print(f'Specificity: {specificity}')

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)

# Crear la matriz de confusión gráfica
plt.figure(figsize=(6, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

## Clasificación cancer con GEMA y Borderline SMOTE

In [ ]:
main_classification_bsmote_tomek = GEMA.Classification(main_map, data_bsmote_tomek)
print("bsmote+tommed quantification:",main_classification_bsmote_tomek.quantization_error)
print("bsmote+tommed topological:",main_classification_bsmote_tomek.topological_error)

main_classification_bsmote_edited= GEMA.Classification(main_map, data_bsmote_edited)
print("bsmote+edited quantification:",main_classification_bsmote_edited.quantization_error)
print("bsmote+edited topological:",main_classification_bsmote_edited.topological_error)

main_classification_bsmote_cnn= GEMA.Classification(main_map, data_bsmote_cnn)
print("bsmote+cnn quantification:",main_classification_bsmote_cnn.quantization_error)
print("bsmote+cnn topological:",main_classification_bsmote_cnn.topological_error)

main_classification_bsmote_ncr= GEMA.Classification(main_map, data_bsmote_ncr)
print("bsmote+ncr quantification:",main_classification_bsmote_ncr.quantization_error)
print("bsmote+ncr topological:",main_classification_bsmote_ncr.topological_error)

main_classification_bsmote_oss= GEMA.Classification(main_map, data_bsmote_oss)
print("bsmote+oss quantification:",main_classification_bsmote_oss.quantization_error)
print("bsmote+oss topological:",main_classification_bsmote_oss.topological_error)

In [ ]:
def smote_enn_clase0(dataset, target):

    print(dataset.head(5))
    print(dataset[3].value_counts())   

    # Separar las características y el target
    X = dataset.drop(target, axis=1)
    y = dataset[target]

    # Guardar los índices de las filas originales
    original_indices = X.index

    # Crear una instancia de SMOTE
    smote = BorderlineSMOTE(sampling_strategy={0: int(len(y[y==0])*1.7), 1: len(y[y==1])})

    # Crear una instancia de Condensed Nearest Neighbour
    enn = TomekLinks(sampling_strategy='all')
    pipeline = Pipeline([('BSMOTE', smote), ('TMK', enn)])

    X_resampled, y_resampled = pipeline.fit_resample(X, y)

    # Redimensionamos las clases.
    y_resampled = y_resampled.to_numpy().reshape(-1, 1)
    y_resampled = y_resampled.reshape(np.size(X_resampled, 0),1)
    
    cols_df = dataset.columns
    cols_df = cols_df.drop(target)
    
    synthetic_data = pd.DataFrame(X_resampled, columns=cols_df)
    synthetic_data[target] = y_resampled

    # Eliminar las filas del conjunto de datos sobremuestreado que corresponden a las filas originales
    synthetic_data_only = synthetic_data.drop(original_indices)
                
    return synthetic_data_only

def smote_enn_clase1(dataset, target):

    print(dataset.head(5))
    print(dataset[3].value_counts())   

    # Separar las características y el target
    X = dataset.drop(target, axis=1)
    y = dataset[target]

    # Guardar los índices de las filas originales
    original_indices = X.index

    # Crear una instancia de SMOTE
    smote = BorderlineSMOTE(sampling_strategy='auto')

    # Crear una instancia de Condensed Nearest Neighbour
    enn = TomekLinks(sampling_strategy='all')
    pipeline = Pipeline([('BSMOTE', smote), ('TMK', enn)])

    X_resampled, y_resampled = pipeline.fit_resample(X, y)

    # Redimensionamos las clases.
    y_resampled = y_resampled.to_numpy().reshape(-1, 1)
    y_resampled = y_resampled.reshape(np.size(X_resampled, 0),1)
    
    cols_df = dataset.columns
    cols_df = cols_df.drop(target)
    
    synthetic_data = pd.DataFrame(X_resampled, columns=cols_df)
    synthetic_data[target] = y_resampled

    # Eliminar las filas del conjunto de datos sobremuestreado que corresponden a las filas originales
    synthetic_data_only = synthetic_data.drop(original_indices)
                
    return synthetic_data_only

Smote_enn_sinteticos0=smote_enn_clase0(data_std,3)
Smote_enn_sinteticos1=smote_enn_clase1(data_std,3)
smote_enn_sinteticos = pd.concat([Smote_enn_sinteticos0, Smote_enn_sinteticos1], axis=0, ignore_index=True)
print(smote_enn_sinteticos.head(5))
print(smote_enn_sinteticos[3].value_counts())

In [ ]:
y = smote_enn_sinteticos.iloc[: , 3]
X = smote_enn_sinteticos.drop(smote_enn_sinteticos.columns[3], axis=1)
y = y.to_numpy()
X = X.to_numpy()
porcentaje = 0.8

# Inicializa un mapa de colores del tamaño del SOM con todos los valores en blanco
color_map = np.ones((9, 9, 3))

# Inicializa un mapa de clases del tamaño del SOM con todos los valores en -1
class_map = -1 * np.ones((9, 9))

# Contadores para cada neurona
counts = np.zeros((9, 9, 2), dtype=int)

# Itera sobre cada dato y su clase correspondiente
for datum, class_ in zip(X, y):
    # Encuentra la neurona ganadora para este dato
    winner = main_map.calculate_bmu(datum)[1]
    
    # Actualiza el contador de la clase para la neurona ganadora
    counts[int(winner[0]), int(winner[1]), int(class_)] += 1
    
    # Actualiza el mapa de clases con la clase reconocida
    class_map[winner] = class_

# Itera sobre cada neurona para determinar el color final
for i in range(9):
    for j in range(9):
        # Calcula el porcentaje de cada clase para la neurona actual
        percent_class_0 = counts[i, j, 0] / (counts[i, j, 0] + counts[i, j, 1]) if counts[i, j, 0] + counts[i, j, 1] > 0 else 0
        percent_class_1 = counts[i, j, 1] / (counts[i, j, 0] + counts[i, j, 1]) if counts[i, j, 0] + counts[i, j, 1] > 0 else 0
        
        # Asigna el color según las condiciones especificadas
        if percent_class_0 >= porcentaje:
            color_map[i, j] = [0, 0, 1]  # Azul
        elif percent_class_1 >= porcentaje:
            color_map[i, j] = [1, 0, 0]  # Rojo
        elif counts[i, j, 0] + counts[i, j, 1] == 0:
            color_map[i, j] = [1, 1, 1]  # Blanco si no ha reconocido ningún patrón
        else:
            color_map[i, j] = [0, 1, 0]  # Verde si no alcanza el 80% de ninguna clase

# Crea una figura con diferentes tamaños para los subplots
fig = plt.figure(figsize=(8, 6))

# Crea una leyenda para el primer gráfico
legend_elements = [Patch(facecolor='blue', edgecolor='b', label='Clase 0'),
                   Patch(facecolor='red', edgecolor='r', label='Clase 1'),
                   Patch(facecolor='green', edgecolor='g', label='Ambas clases')]
ax1 = fig.add_subplot(121)  # 121 significa "1 fila, 2 columnas, primer gráfico"
#ax1.legend(handles=legend_elements, loc='upper right')
ax1.imshow(np.rot90(color_map))
ax1.set_title('Mapa de neuronas w/o escala')

# Inicializa un mapa de neuronas con ceros
neuron_map = np.zeros((9, 9))
ganadores = np.zeros((9, 9))

# Itera sobre los datos y las clases
for datum, clss in zip(X, y):
    # Obtiene la neurona ganadora para el dato actual
    winner = main_map.calculate_bmu(datum)[1]
    ganadores[winner] = 1
    print(winner, clss)
    # Si la clase es 0, resta 1, si la clase es 1, suma 1
    neuron_map[winner[0], winner[1]] += 1 if clss == 1 else -1

# Normaliza el mapa de neuronas para que los valores estén entre -1 y 1
# neuron_map = neuron_map / np.max(np.abs(neuron_map))

# Define los colores de la escala como azul, verde y amarillo
cmap_colors = ["indigo", "royalblue", "limegreen", "yellow"]

# Crea el mapa de colores
cmap = LinearSegmentedColormap.from_list("mycmap", cmap_colors)

# Dibuja el mapa de calor
ax2 = fig.add_subplot(122)  # 122 significa "1 fila, 2 columnas, segundo gráfico"
im = ax2.imshow(np.rot90(neuron_map), cmap=cmap, interpolation='nearest')
ax2.set_title('Mapa de neuronas con escala')

# Muestra la barra de colores
fig.colorbar(im, ax=ax2)

# Ajusta el layout
plt.tight_layout()

# Muestra el gráfico
plt.show()

data_balanced_dropped_smote_edited_sint = smote_enn_sinteticos.drop(columns = [3])
data_balanced_dropped_smote_edited_sint=data_balanced_dropped_smote_edited_sint.to_numpy()
main_classification_smoted_enn_sinteticos = GEMA.Classification(main_map, data_balanced_dropped_smote_edited_sint)
GEMA.Visualization.heat_map(main_classification_smoted_enn_sinteticos)

## Clasificación cancer con Minisom y Borderline SMOTE

In [ ]:
print("bsmote+tommed")
print(som.quantization_error(data_bsmote_tomek), som.topographic_error(data_bsmote_tomek))
print("bsmote+linked")
print(som.quantization_error(data_bsmote_edited), som.topographic_error(data_bsmote_edited))
print("bsmote+cnn")
print(som.quantization_error(data_bsmote_cnn), som.topographic_error(data_bsmote_cnn))
print("bsmote+ncr")
print(som.quantization_error(data_bsmote_ncr), som.topographic_error(data_bsmote_ncr))
print("bsmote+oss")
print(som.quantization_error(data_bsmote_oss), som.topographic_error(data_bsmote_oss))

## SVM SMOTE Cancer

In [ ]:
## SVMSMOTE + Tomed_links
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 3)
data_std = wf.min_max_normalization(datos_mode)

svmsmote_tomek=wf.SVMSMOTE_tomek(data_std,3)
print(svmsmote_tomek[3].value_counts())
data_svmsmote_tomek = svmsmote_tomek.drop(columns = [3])
data_svmsmote_tomek=data_svmsmote_tomek.to_numpy()

## SVMSMOTE + ENN
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 3)
data_std = wf.min_max_normalization(datos_mode)

svmsmote_edited=wf.SVMSMOTE_enn(data_std,3)
print(svmsmote_edited[3].value_counts())
data_svmsmote_edited = svmsmote_edited.drop(columns = [3])
data_svmsmote_edited=data_svmsmote_edited.to_numpy()

## SVMSMOTE con CNN (condensed nearest neighbors)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 3)
data_std = wf.min_max_normalization(datos_mode)

svmsmote_cnn=wf.SVMSMOTE_cnn(data_std,3)
print(svmsmote_cnn[3].value_counts())
data_svmsmote_cnn = svmsmote_cnn.drop(columns = [3])
data_svmsmote_cnn=data_svmsmote_cnn.to_numpy()

## SVMSMOTE con NCR (Neighboorhood cleaning rule)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 3)
data_std = wf.min_max_normalization(datos_mode)

svmsmote_ncr=wf.SVMSMOTE_ncr(data_std,3)
print(svmsmote_ncr[3].value_counts())
data_svmsmote_ncr = svmsmote_ncr.drop(columns = [3])
data_svmsmote_ncr=data_svmsmote_ncr.to_numpy()

## SVMSMOTE con OSD (One sided selection)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 3)
data_std = wf.min_max_normalization(datos_mode)

svmsmote_oss=wf.SVMSMOTE_oss(data_std,3)
print(svmsmote_oss[3].value_counts())
data_svmsmote_oss = svmsmote_oss.drop(columns = [3])
data_svmsmote_oss=data_svmsmote_oss.to_numpy()

In [ ]:
y = svmsmote_tomek.iloc[: , 3]
X = svmsmote_tomek.drop(svmsmote_tomek.columns[3], axis=1)

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

# Dividir el conjunto de entrenamiento en conjunto de entrenamiento y conjunto de validación
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, shuffle=True)

# Escalar las características para mejorar el rendimiento del modelo
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valn = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Crear un modelo secuencial en Keras
model = Sequential()
model.add(Dense(units=45, activation='relu', input_dim=3))  # Capa de entrada con 20 características
model.add(Dense(units=45, activation='relu'))  # Capa oculta con 32 neuronas y función de activación ReLU
model.add(Dense(units=1, activation='sigmoid'))  # Capa de salida con una neurona y función de activación sigmoide
pesos = model.get_weights()
# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=25, batch_size=32, validation_data=(X_val, y_val))

# Calcular Accuracy en el conjunto de entrenamiento
train_accuracy = accuracy_score(y_train, model.predict(X_train) > 0.5)

# Calcular Accuracy en el conjunto de validación
validation_accuracy = accuracy_score(y_val, model.predict(X_val) > 0.5)

print(f'Training Accuracy: {train_accuracy}')
print(f'Validation Accuracy: {validation_accuracy}')

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)  # Convertir las probabilidades en etiquetas binarias (0 o 1)

# Calcular métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)

# Calcular la especificidad
TN = conf_matrix[0][0]  # Verdaderos negativos
FP = conf_matrix[0][1]  # Falsos positivos
specificity = TN / (TN + FP)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall (Sensibility): {recall}')
print(f'Specificity: {specificity}')

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)

# Crear la matriz de confusión gráfica
plt.figure(figsize=(6, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


In [ ]:
y = svmsmote_oss.iloc[: , 3]
X = svmsmote_oss.drop(svmsmote_oss.columns[3], axis=1)

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

# Dividir el conjunto de entrenamiento en conjunto de entrenamiento y conjunto de validación
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, shuffle=True)

# Escalar las características para mejorar el rendimiento del modelo
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valn = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Crear un modelo secuencial en Keras
model = Sequential()
model.add(Dense(units=45, activation='relu', input_dim=3))  # Capa de entrada con 20 características
model.add(Dense(units=45, activation='relu'))  # Capa oculta con 32 neuronas y función de activación ReLU
model.add(Dense(units=1, activation='sigmoid'))  # Capa de salida con una neurona y función de activación sigmoide
model.set_weights(pesos)
# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=25, batch_size=32, validation_data=(X_val, y_val))

# Calcular Accuracy en el conjunto de entrenamiento
train_accuracy = accuracy_score(y_train, model.predict(X_train) > 0.5)

# Calcular Accuracy en el conjunto de validación
validation_accuracy = accuracy_score(y_val, model.predict(X_val) > 0.5)

print(f'Training Accuracy: {train_accuracy}')
print(f'Validation Accuracy: {validation_accuracy}')

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)  # Convertir las probabilidades en etiquetas binarias (0 o 1)

# Calcular métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)

# Calcular la especificidad
TN = conf_matrix[0][0]  # Verdaderos negativos
FP = conf_matrix[0][1]  # Falsos positivos
specificity = TN / (TN + FP)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall (Sensibility): {recall}')
print(f'Specificity: {specificity}')

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)

# Crear la matriz de confusión gráfica
plt.figure(figsize=(6, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

## Clasificación cancer con GEMA y SVM SMOTE

In [ ]:
main_classification_svmsmote_tomek = GEMA.Classification(main_map, data_svmsmote_tomek)
print("svmsmote+tommed quantification:",main_classification_svmsmote_tomek.quantization_error)
print("svmsmote+tommed topological:",main_classification_svmsmote_tomek.topological_error)

main_classification_svmsmote_edited= GEMA.Classification(main_map, data_svmsmote_edited)
print("svmsmote+edited quantification:",main_classification_svmsmote_edited.quantization_error)
print("svmsmote+edited topological:",main_classification_svmsmote_edited.topological_error)

main_classification_svmsmote_cnn= GEMA.Classification(main_map, data_svmsmote_cnn)
print("svmsmote+cnn quantification:",main_classification_svmsmote_cnn.quantization_error)
print("svmsmote+cnn topological:",main_classification_svmsmote_cnn.topological_error)

main_classification_svmsmote_ncr= GEMA.Classification(main_map, data_svmsmote_ncr)
print("svmsmote+ncr quantification:",main_classification_svmsmote_ncr.quantization_error)
print("svmsmote+ncr topological:",main_classification_svmsmote_ncr.topological_error)

main_classification_svmsmote_oss= GEMA.Classification(main_map, data_svmsmote_oss)
print("svmsmote+oss quantification:",main_classification_svmsmote_oss.quantization_error)
print("svmsmote+oss topological:",main_classification_svmsmote_oss.topological_error)

## Clasificación cancer con Minisom y SVM SMOTE

In [ ]:
print("svmsmote+tommed")
print(som.quantization_error(data_svmsmote_tomek), som.topographic_error(data_svmsmote_tomek))
print("svmsmote+linked")
print(som.quantization_error(data_svmsmote_edited), som.topographic_error(data_svmsmote_edited))
print("svmsmote+cnn")
print(som.quantization_error(data_svmsmote_cnn), som.topographic_error(data_svmsmote_cnn))
print("svmsmote+ncr")
print(som.quantization_error(data_svmsmote_ncr), som.topographic_error(data_svmsmote_ncr))
print("svmsmote+oss")
print(som.quantization_error(data_svmsmote_oss), som.topographic_error(data_svmsmote_oss))

## Kmeans SMOTE cancer

In [ ]:
## KMSSMOTE + Tomed_links
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 3)
data_std = wf.min_max_normalization(datos_mode)

kmssmote_tomek=wf.KMSSMOTE_tomek(data_std,3)
print(kmssmote_tomek[3].value_counts())
data_kmssmote_tomek = kmssmote_tomek.drop(columns = [3])
data_kmssmote_tomek=data_kmssmote_tomek.to_numpy()

## KMSSMOTE + ENN
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 3)
data_std = wf.min_max_normalization(datos_mode)

kmssmote_edited=wf.KMSSMOTE_enn(data_std,3)
print(kmssmote_edited[3].value_counts())
data_kmssmote_edited = kmssmote_edited.drop(columns = [3])
data_kmssmote_edited=data_kmssmote_edited.to_numpy()

## KMSSMOTE con CNN (condensed nearest neighbors)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 3)
data_std = wf.min_max_normalization(datos_mode)

kmssmote_cnn=wf.KMSSMOTE_cnn(data_std,3)
print(kmssmote_cnn[3].value_counts())
data_kmssmote_cnn = kmssmote_cnn.drop(columns = [3])
data_kmssmote_cnn=data_kmssmote_cnn.to_numpy()

## KMSSMOTE con NCR (Neighboorhood cleaning rule)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 3)
data_std = wf.min_max_normalization(datos_mode)

kmssmote_ncr=wf.KMSSMOTE_ncr(data_std,3)
print(kmssmote_ncr[3].value_counts())
data_kmssmote_ncr = kmssmote_ncr.drop(columns = [3])
data_kmssmote_ncr=data_kmssmote_ncr.to_numpy()

## KMSSMOTE con OSD (One sided selection)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 3)
data_std = wf.min_max_normalization(datos_mode)

kmssmote_oss=wf.KMSSMOTE_oss(data_std,3)
print(kmssmote_oss[3].value_counts())
data_kmssmote_oss = kmssmote_oss.drop(columns = [3])
data_kmssmote_oss=data_kmssmote_oss.to_numpy()

In [ ]:
y = kmssmote_tomek.iloc[: , 3]
X = kmssmote_tomek.drop(kmssmote_tomek.columns[3], axis=1)

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

# Dividir el conjunto de entrenamiento en conjunto de entrenamiento y conjunto de validación
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, shuffle=True)

# Escalar las características para mejorar el rendimiento del modelo
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valn = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Crear un modelo secuencial en Keras
model = Sequential()
model.add(Dense(units=45, activation='relu', input_dim=3))  # Capa de entrada con 20 características
model.add(Dense(units=45, activation='relu'))  # Capa oculta con 32 neuronas y función de activación ReLU
model.add(Dense(units=1, activation='sigmoid'))  # Capa de salida con una neurona y función de activación sigmoide
pesos = model.get_weights()
# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=25, batch_size=32, validation_data=(X_val, y_val))

# Calcular Accuracy en el conjunto de entrenamiento
train_accuracy = accuracy_score(y_train, model.predict(X_train) > 0.5)

# Calcular Accuracy en el conjunto de validación
validation_accuracy = accuracy_score(y_val, model.predict(X_val) > 0.5)

print(f'Training Accuracy: {train_accuracy}')
print(f'Validation Accuracy: {validation_accuracy}')

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)  # Convertir las probabilidades en etiquetas binarias (0 o 1)

# Calcular métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)

# Calcular la especificidad
TN = conf_matrix[0][0]  # Verdaderos negativos
FP = conf_matrix[0][1]  # Falsos positivos
specificity = TN / (TN + FP)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall (Sensibility): {recall}')
print(f'Specificity: {specificity}')

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)

# Crear la matriz de confusión gráfica
plt.figure(figsize=(6, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
y = kmssmote_oss.iloc[: , 3]
X = kmssmote_oss.drop(kmssmote_oss.columns[3], axis=1)

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

# Dividir el conjunto de entrenamiento en conjunto de entrenamiento y conjunto de validación
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, shuffle=True)

# Escalar las características para mejorar el rendimiento del modelo
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valn = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Crear un modelo secuencial en Keras
model = Sequential()
model.add(Dense(units=45, activation='relu', input_dim=3))  # Capa de entrada con 20 características
model.add(Dense(units=45, activation='relu'))  # Capa oculta con 32 neuronas y función de activación ReLU
model.add(Dense(units=1, activation='sigmoid'))  # Capa de salida con una neurona y función de activación sigmoide
model.set_weights(pesos)
# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=25, batch_size=32, validation_data=(X_val, y_val))

# Calcular Accuracy en el conjunto de entrenamiento
train_accuracy = accuracy_score(y_train, model.predict(X_train) > 0.5)

# Calcular Accuracy en el conjunto de validación
validation_accuracy = accuracy_score(y_val, model.predict(X_val) > 0.5)

print(f'Training Accuracy: {train_accuracy}')
print(f'Validation Accuracy: {validation_accuracy}')

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)  # Convertir las probabilidades en etiquetas binarias (0 o 1)

# Calcular métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)

# Calcular la especificidad
TN = conf_matrix[0][0]  # Verdaderos negativos
FP = conf_matrix[0][1]  # Falsos positivos
specificity = TN / (TN + FP)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall (Sensibility): {recall}')
print(f'Specificity: {specificity}')

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)

# Crear la matriz de confusión gráfica
plt.figure(figsize=(6, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

## Clasificación cancer con GEMA y Kmeans SMOTE

In [ ]:
main_classification_kmssmote_tomek = GEMA.Classification(main_map, data_kmssmote_tomek)
print("svmsmote+tommed quantification:",main_classification_kmssmote_tomek.quantization_error)
print("svmsmote+tommed topological:",main_classification_kmssmote_tomek.topological_error)

main_classification_kmssmote_edited= GEMA.Classification(main_map, data_kmssmote_edited)
print("svmsmote+edited quantification:",main_classification_kmssmote_edited.quantization_error)
print("svmsmote+edited topological:",main_classification_kmssmote_edited.topological_error)

main_classification_kmssmote_cnn= GEMA.Classification(main_map, data_kmssmote_cnn)
print("svmsmote+cnn quantification:",main_classification_kmssmote_cnn.quantization_error)
print("svmsmote+cnn topological:",main_classification_kmssmote_cnn.topological_error)

main_classification_kmssmote_ncr= GEMA.Classification(main_map, data_kmssmote_ncr)
print("svmsmote+ncr quantification:",main_classification_kmssmote_ncr.quantization_error)
print("svmsmote+ncr topological:",main_classification_kmssmote_ncr.topological_error)

main_classification_kmssmote_oss= GEMA.Classification(main_map, data_kmssmote_oss)
print("svmsmote+oss quantification:",main_classification_kmssmote_oss.quantization_error)
print("svmsmote+oss topological:",main_classification_kmssmote_oss.topological_error)

## Clasificación cancer con Minisom y Kmeans SMOTE

In [ ]:
print("kmssmote+tommed")
print(som.quantization_error(data_kmssmote_tomek), som.topographic_error(data_kmssmote_tomek))
print("kmssmote+linked")
print(som.quantization_error(data_kmssmote_edited), som.topographic_error(data_kmssmote_edited))
print("kmssmote+cnn")
print(som.quantization_error(data_kmssmote_cnn), som.topographic_error(data_kmssmote_cnn))
print("kmssmote+ncr")
print(som.quantization_error(data_kmssmote_ncr), som.topographic_error(data_kmssmote_ncr))
print("kmssmote+oss")
print(som.quantization_error(data_kmssmote_oss), som.topographic_error(data_kmssmote_oss))

## Importar dataset de creditos bancarios


In [32]:
## Importar dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/german.csv'
missing_values = [' ', 'NaN', 'na', 'Na', '-', '--', 'n/a', '?']
columns = ['class']
data = pd.read_csv(url, na_values = missing_values, header=None)
columns_to_encode = [0, 2, 3, 4, 5, 6, 8, 9, 11, 13, 14, 16, 18, 19]

# Inicializar el LabelEncoder
label_encoder = LabelEncoder()

# Aplicar LabelEncoder a las columnas seleccionadas
for column in columns_to_encode:
    data[column] = label_encoder.fit_transform(data[column])

data[20] = data[20].replace({1: 0, 2: 1})

In [ ]:
data

In [ ]:
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = data_dropped
datos_mode = wf.mode_imputation(data_dropped, 3)
datos_mode[20].value_counts()


In [ ]:
datos_mode=datos_mode.drop(columns = [20])
df = datos_mode.astype(int, errors='ignore')
print(df)
data_std = wf.min_max_normalization(datos_mode)
target = data[20].values
label_names={0:'malo', 1:'bueno'}
data_som=data_std.to_numpy()

In [ ]:
data_som

## Entrenamiento GEMA créditos

In [ ]:
main_map = GEMA.Map(data_som, 
                    13, 
                    5000, 
                    initial_lr=0.3, 
                    initial_neighbourhood = 0,
                    distance = 'euclidean',
                    use_decay= False,
                    presentation= 'random',
                    weights = 'random')

main_classification = GEMA.Classification(main_map, data_som)
print("quantification error:",main_classification.quantization_error)
print("Topological_error:",main_classification.topological_error)

In [ ]:
#import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.colors import LinearSegmentedColormap

porcentaje = 0.8

# Inicializa un mapa de colores del tamaño del SOM con todos los valores en blanco
color_map = np.ones((13, 13, 3))

# Inicializa un mapa de clases del tamaño del SOM con todos los valores en -1
class_map = -1 * np.ones((13, 13))

# Contadores para cada neurona
counts = np.zeros((13, 13, 2), dtype=int)

# Itera sobre cada dato y su clase correspondiente
for datum, class_ in zip(data_som, target):
    # Encuentra la neurona ganadora para este dato
    winner = main_map.calculate_bmu(datum)[1]
    
    # Actualiza el contador de la clase para la neurona ganadora
    counts[int(winner[0]), int(winner[1]), int(class_)] += 1
    
    # Actualiza el mapa de clases con la clase reconocida
    class_map[winner] = class_

# Itera sobre cada neurona para determinar el color final
for i in range(13):
    for j in range(13):
        # Calcula el porcentaje de cada clase para la neurona actual
        percent_class_0 = counts[i, j, 0] / (counts[i, j, 0] + counts[i, j, 1]) if counts[i, j, 0] + counts[i, j, 1] > 0 else 0
        percent_class_1 = counts[i, j, 1] / (counts[i, j, 0] + counts[i, j, 1]) if counts[i, j, 0] + counts[i, j, 1] > 0 else 0
        
        # Asigna el color según las condiciones especificadas
        if percent_class_0 >= porcentaje:
            color_map[i, j] = [0, 0, 1]  # Azul
        elif percent_class_1 >= porcentaje:
            color_map[i, j] = [1, 0, 0]  # Rojo
        elif counts[i, j, 0] + counts[i, j, 1] == 0:
            color_map[i, j] = [1, 1, 1]  # Blanco si no ha reconocido ningún patrón
        else:
            color_map[i, j] = [0, 1, 0]  # Verde si no alcanza el 80% de ninguna clase


# Count the number of neurons of each color
num_blue = np.sum(np.all(color_map == [0, 0, 1], axis=-1))
num_red = np.sum(np.all(color_map == [1, 0, 0], axis=-1))
num_white = np.sum(np.all(color_map == [1, 1, 1], axis=-1))
num_green = np.sum(np.all(color_map == [0, 1, 0], axis=-1))

# Crea una figura con diferentes tamaños para los subplots
fig = plt.figure(figsize=(8, 6))

# Crea una leyenda para el primer gráfico
legend_elements = [Patch(facecolor='blue', edgecolor='b', label='Clase 0'),
                   Patch(facecolor='red', edgecolor='r', label='Clase 1'),
                   Patch(facecolor='green', edgecolor='g', label='Ambas clases')]
ax1 = fig.add_subplot(121)  # 121 significa "1 fila, 2 columnas, primer gráfico"
# ax1.legend(handles=legend_elements, loc='upper right')
ax1.imshow(np.rot90(color_map))
ax1.set_title('Mapa de neuronas w/o escala')

# Inicializa un mapa de neuronas con ceros
neuron_map = np.zeros((13, 13))
ganadores = np.zeros((13, 13))

# Itera sobre los datos y las clases
for datum, clss in zip(data_som, target):
    # Obtiene la neurona ganadora para el dato actual
    winner = main_map.calculate_bmu(datum)[1]
    ganadores[winner] = 1
    print(winner, clss)
    # Si la clase es 0, resta 1, si la clase es 1, suma 1
    neuron_map[winner[0], winner[1]] += 1 if clss == 1 else -1

# Normaliza el mapa de neuronas para que los valores estén entre -1 y 1
# neuron_map = neuron_map / np.max(np.abs(neuron_map))

# Define los colores de la escala como azul, verde y amarillo
cmap_colors = ["indigo", "royalblue", "limegreen", "yellow"]

# Crea el mapa de colores
cmap = LinearSegmentedColormap.from_list("mycmap", cmap_colors)

# Dibuja el mapa de calor
ax2 = fig.add_subplot(122)  # 122 significa "1 fila, 2 columnas, segundo gráfico"
im = ax2.imshow(np.rot90(neuron_map), cmap=cmap, interpolation='nearest')
ax2.set_title('Mapa de neuronas con escala')

# Muestra la barra de colores
fig.colorbar(im, ax=ax2)

# Ajusta el layout
plt.tight_layout()

# Muestra el gráfico
plt.show()

# Print the number of neurons of each color
print(f"Number of blue neurons: {num_blue}")
print(f"Number of red neurons: {num_red}")
print(f"Number of white neurons: {num_white}")
print(f"Number of green neurons: {num_green}")

In [ ]:
#GEMA.Visualization.codebook_vectors(main_map, np.array(["Not Spiled", "Spiled"]))

## Entrenamiento Minisom créditos

In [ ]:
n_neurons = 17
m_neurons = 17

som = MiniSom(n_neurons, m_neurons, data_som.shape[1], learning_rate=0.27199933063716647, random_seed=42)
som.train(data_som, num_iteration=9400, verbose=True)  # random training
print("Topological error:",som.topographic_error(data_som))

## SMOTE créditos

In [ ]:
## Smote + Tomed_links
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 20)
data_std = wf.min_max_normalization(datos_mode)

Smote_tomed=wf.smote_tomed_link(data_std,20)
print(Smote_tomed[20].value_counts())
# Smote_tomed= Smote_tomed.reset_index(drop=True, inplace=True)
# data_std= data_std.reset_index(drop=True, inplace=True)
# diferentes=Smote_tomed[(Smote_tomed == data_std).all(axis=1) == False]
# print(diferentes)
data_balanced_dropped_smote_tomed = Smote_tomed.drop(columns = [20])
data_balanced_dropped_smote_tomed=data_balanced_dropped_smote_tomed.to_numpy()

## smote_edited_nearest_neighbor
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 20)
data_std = wf.min_max_normalization(datos_mode)

Smote_edited=wf.smote_edited_nearest_neighbor(data_std,20)
print(Smote_edited[20].value_counts())
data_balanced_dropped_smote_edited = Smote_edited.drop(columns = [20])
data_balanced_dropped_smote_edited=data_balanced_dropped_smote_edited.to_numpy()

## smote con CNN (condensed nearest neighbors)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 20)
data_std = wf.min_max_normalization(datos_mode)

Smote_cnn=wf.smote_cnn(data_std,20)
print(Smote_cnn[20].value_counts())
data_smote_cnn = Smote_cnn.drop(columns = [20])
data_smote_cnn=data_smote_cnn.to_numpy()

## smote con NCR (Neighboorhood cleaning rule)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 20)
data_std = wf.min_max_normalization(datos_mode)

Smote_ncr=wf.smote_ncr(data_std,20)
print(Smote_ncr[20].value_counts())
data_smote_ncr = Smote_ncr.drop(columns = [20])
data_smote_ncr=data_smote_ncr.to_numpy()

## smote con OSD (One sided selection)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 20)
data_std = wf.min_max_normalization(datos_mode)

Smote_oss=wf.smote_osd(data_std,20)
print(Smote_oss[20].value_counts())
data_smote_oss = Smote_oss.drop(columns = [20])
data_smote_oss=data_smote_oss.to_numpy()

## Clasificación créditos con GEMA y SMOTE

In [ ]:
main_classification_smoted_tomed = GEMA.Classification(main_map, data_balanced_dropped_smote_tomed)
print("Smote+tommed quantification:",main_classification_smoted_tomed.quantization_error)
print("Smote+tommed topological:",main_classification_smoted_tomed.topological_error)

main_classification_smoted_edited= GEMA.Classification(main_map, data_balanced_dropped_smote_edited)
print("Smote+edited quantification:",main_classification_smoted_edited.quantization_error)
print("Smote+edited topological:",main_classification_smoted_edited.topological_error)

main_classification_smoted_cnn= GEMA.Classification(main_map, data_smote_cnn)
print("Smote+cnn quantification:",main_classification_smoted_cnn.quantization_error)
print("Smote+cnn topological:",main_classification_smoted_cnn.topological_error)

main_classification_smoted_ncr= GEMA.Classification(main_map, data_smote_ncr)
print("Smote+ncr quantification:",main_classification_smoted_ncr.quantization_error)
print("Smote+ncr topological:",main_classification_smoted_ncr.topological_error)

main_classification_smoted_oss= GEMA.Classification(main_map, data_smote_oss)
print("Smote+oss quantification:",main_classification_smoted_oss.quantization_error)
print("Smote+oss topological:",main_classification_smoted_oss.topological_error)

## Clasificación créditos con Minisom y SMOTE

In [ ]:
print("Smote+tommed")
print(som.quantization_error(data_balanced_dropped_smote_tomed), som.topographic_error(data_balanced_dropped_smote_tomed))
print("Smote+linked")
print(som.quantization_error(data_balanced_dropped_smote_edited), som.topographic_error(data_balanced_dropped_smote_edited))
print("Smote+cnn")
print(som.quantization_error(data_smote_cnn), som.topographic_error(data_smote_cnn))
print("Smote+ncr")
print(som.quantization_error(data_smote_ncr), som.topographic_error(data_smote_ncr))
print("Smote+oss")
print(som.quantization_error(data_smote_oss), som.topographic_error(data_smote_oss))

## ADASYN créditos

In [ ]:
## ADASYN + Tomed_links
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 20)
data_std = wf.min_max_normalization(datos_mode)

adasyn_tomek=wf.adasyn_tomek(data_std,20)
print(adasyn_tomek[20].value_counts())
data_adasyn_tomek = adasyn_tomek.drop(columns = [20])
data_adasyn_tomek=data_adasyn_tomek.to_numpy()

## ADASYN + ENN
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 20)
data_std = wf.min_max_normalization(datos_mode)

adasyn_edited=wf.adasyn_enn(data_std,20)
print(adasyn_edited[20].value_counts())
data_adasyn_edited = adasyn_edited.drop(columns = [20])
data_adasyn_edited=data_adasyn_edited.to_numpy()

## ADASYN con CNN (condensed nearest neighbors)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 20)
data_std = wf.min_max_normalization(datos_mode)

adasyn_cnn=wf.adasyn_cnn(data_std,20)
print(adasyn_cnn[20].value_counts())
data_adasyn_cnn = adasyn_cnn.drop(columns = [20])
data_adasyn_cnn=data_adasyn_cnn.to_numpy()

## ADASYN con NCR (Neighboorhood cleaning rule)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 20)
data_std = wf.min_max_normalization(datos_mode)

adasyn_ncr=wf.adasyn_ncr(data_std,20)
print(adasyn_ncr[20].value_counts())
data_adasyn_ncr = adasyn_ncr.drop(columns = [20])
data_adasyn_ncr=data_adasyn_ncr.to_numpy()

## ADASYN con OSD (One sided selection)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 20)
data_std = wf.min_max_normalization(datos_mode)

adasyn_oss=wf.adasyn_oss(data_std,20)
print(adasyn_oss[20].value_counts())
data_adasyn_oss = adasyn_oss.drop(columns = [20])
data_adasyn_oss=data_adasyn_oss.to_numpy()

## Clasificación créditos con GEMA y ADASYN

In [ ]:
main_classification_adasyn_tomek = GEMA.Classification(main_map, data_adasyn_tomek)
print("adasyn+tommed quantification:",main_classification_adasyn_tomek.quantization_error)
print("adasyn+tommed topological:",main_classification_adasyn_tomek.topological_error)

main_classification_adasyn_edited= GEMA.Classification(main_map, data_adasyn_edited)
print("adasyn+edited quantification:",main_classification_adasyn_edited.quantization_error)
print("adasyn+edited topological:",main_classification_adasyn_edited.topological_error)

main_classification_smoted_cnn= GEMA.Classification(main_map, data_smote_cnn)
print("adasyn+cnn quantification:",main_classification_smoted_cnn.quantization_error)
print("adasyn+cnn topological:",main_classification_smoted_cnn.topological_error)

main_classification_smoted_ncr= GEMA.Classification(main_map, data_smote_ncr)
print("adasyn+ncr quantification:",main_classification_smoted_ncr.quantization_error)
print("adasyn+ncr topological:",main_classification_smoted_ncr.topological_error)

main_classification_smoted_oss= GEMA.Classification(main_map, data_smote_oss)
print("adasyn+oss quantification:",main_classification_smoted_oss.quantization_error)
print("adasyn+oss topological:",main_classification_smoted_oss.topological_error)

## Clasificación créditos con Minisom y ADASYN

In [ ]:
print("adasyn+tommed")
print(som.quantization_error(data_adasyn_tomek), som.topographic_error(data_adasyn_tomek))
print("adasyn+linked")
print(som.quantization_error(data_adasyn_edited), som.topographic_error(data_adasyn_edited))
print("adasyn+cnn")
print(som.quantization_error(data_adasyn_cnn), som.topographic_error(data_adasyn_cnn))
print("adasyn+ncr")
print(som.quantization_error(data_adasyn_ncr), som.topographic_error(data_adasyn_ncr))
print("adasyn+oss")
print(som.quantization_error(data_adasyn_oss), som.topographic_error(data_adasyn_oss))

## Borderline SMOTE créditos

In [ ]:
## BSMOTE + Tomed_links
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 20)
data_std = wf.min_max_normalization(datos_mode)

bsmote_tomek=wf.BSMOTE_tomek(data_std,20)
print(bsmote_tomek[20].value_counts())
data_bsmote_tomek = bsmote_tomek.drop(columns = [20])
data_bsmote_tomek=data_bsmote_tomek.to_numpy()

## BSMOTE + ENN
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 20)
data_std = wf.min_max_normalization(datos_mode)

bsmote_edited=wf.BSMOTE_enn(data_std,20)
print(bsmote_edited[20].value_counts())
data_bsmote_edited = bsmote_edited.drop(columns = [20])
data_bsmote_edited=data_bsmote_edited.to_numpy()

## BSMOTE con CNN (condensed nearest neighbors)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 20)
data_std = wf.min_max_normalization(datos_mode)

bsmote_cnn=wf.BSMOTE_cnn(data_std,20)
print(bsmote_cnn[20].value_counts())
data_bsmote_cnn = bsmote_cnn.drop(columns = [20])
data_bsmote_cnn=data_bsmote_cnn.to_numpy()

## BSMOTE con NCR (Neighboorhood cleaning rule)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 20)
data_std = wf.min_max_normalization(datos_mode)

bsmote_ncr=wf.BSMOTE_ncr(data_std,20)
print(bsmote_ncr[20].value_counts())
data_bsmote_ncr = bsmote_ncr.drop(columns = [20])
data_bsmote_ncr=data_bsmote_ncr.to_numpy()

## BSMOTE con OSD (One sided selection)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 20)
data_std = wf.min_max_normalization(datos_mode)

bsmote_oss=wf.BSMOTE_oss(data_std,20)
print(bsmote_oss[20].value_counts())
data_bsmote_oss = bsmote_oss.drop(columns = [20])
data_bsmote_oss=data_bsmote_oss.to_numpy()

## Clasificación créditos con GEMA y Borderline SMOTE

In [ ]:
main_classification_bsmote_tomek = GEMA.Classification(main_map, data_bsmote_tomek)
print("bsmote+tommed quantification:",main_classification_bsmote_tomek.quantization_error)
print("bsmote+tommed topological:",main_classification_bsmote_tomek.topological_error)

main_classification_bsmote_edited= GEMA.Classification(main_map, data_bsmote_edited)
print("bsmote+edited quantification:",main_classification_bsmote_edited.quantization_error)
print("bsmote+edited topological:",main_classification_bsmote_edited.topological_error)

main_classification_bsmote_cnn= GEMA.Classification(main_map, data_bsmote_cnn)
print("bsmote+cnn quantification:",main_classification_bsmote_cnn.quantization_error)
print("bsmote+cnn topological:",main_classification_bsmote_cnn.topological_error)

main_classification_bsmote_ncr= GEMA.Classification(main_map, data_bsmote_ncr)
print("bsmote+ncr quantification:",main_classification_bsmote_ncr.quantization_error)
print("bsmote+ncr topological:",main_classification_bsmote_ncr.topological_error)

main_classification_bsmote_oss= GEMA.Classification(main_map, data_bsmote_oss)
print("bsmote+oss quantification:",main_classification_bsmote_oss.quantization_error)
print("bsmote+oss topological:",main_classification_bsmote_oss.topological_error)

## Clasificación créditos Minisom y Borderline SMOTE

In [ ]:
print("bsmote+tommed")
print(som.quantization_error(data_bsmote_tomek), som.topographic_error(data_bsmote_tomek))
print("bsmote+linked")
print(som.quantization_error(data_bsmote_edited), som.topographic_error(data_bsmote_edited))
print("bsmote+cnn")
print(som.quantization_error(data_bsmote_cnn), som.topographic_error(data_bsmote_cnn))
print("bsmote+ncr")
print(som.quantization_error(data_bsmote_ncr), som.topographic_error(data_bsmote_ncr))
print("bsmote+oss")
print(som.quantization_error(data_bsmote_oss), som.topographic_error(data_bsmote_oss))

## SVM SMOTE créditos

In [ ]:
## SVMSMOTE + Tomed_links
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 20)
data_std = wf.min_max_normalization(datos_mode)

svmsmote_tomek=wf.SVMSMOTE_tomek(data_std,20)
print(svmsmote_tomek[20].value_counts())
data_svmsmote_tomek = svmsmote_tomek.drop(columns = [20])
data_svmsmote_tomek=data_svmsmote_tomek.to_numpy()

## SVMSMOTE + ENN
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 20)
data_std = wf.min_max_normalization(datos_mode)

svmsmote_edited=wf.SVMSMOTE_enn(data_std,20)
print(svmsmote_edited[20].value_counts())
data_svmsmote_edited = svmsmote_edited.drop(columns = [20])
data_svmsmote_edited=data_svmsmote_edited.to_numpy()

## SVMSMOTE con CNN (condensed nearest neighbors)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 20)
data_std = wf.min_max_normalization(datos_mode)

svmsmote_cnn=wf.SVMSMOTE_cnn(data_std,20)
print(svmsmote_cnn[20].value_counts())
data_svmsmote_cnn = svmsmote_cnn.drop(columns = [20])
data_svmsmote_cnn=data_svmsmote_cnn.to_numpy()

## SVMSMOTE con NCR (Neighboorhood cleaning rule)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 20)
data_std = wf.min_max_normalization(datos_mode)

svmsmote_ncr=wf.SVMSMOTE_ncr(data_std,20)
print(svmsmote_ncr[20].value_counts())
data_svmsmote_ncr = svmsmote_ncr.drop(columns = [20])
data_svmsmote_ncr=data_svmsmote_ncr.to_numpy()

## SVMSMOTE con OSD (One sided selection)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 20)
data_std = wf.min_max_normalization(datos_mode)

svmsmote_oss=wf.SVMSMOTE_oss(data_std,20)
print(svmsmote_oss[20].value_counts())
data_svmsmote_oss = svmsmote_oss.drop(columns = [20])
data_svmsmote_oss=data_svmsmote_oss.to_numpy()

## Clasificación créditos GEMA y SVM SMOTE

In [ ]:
main_classification_svmsmote_tomek = GEMA.Classification(main_map, data_svmsmote_tomek)
print("svmsmote+tommed quantification:",main_classification_svmsmote_tomek.quantization_error)
print("svmsmote+tommed topological:",main_classification_svmsmote_tomek.topological_error)

main_classification_svmsmote_edited= GEMA.Classification(main_map, data_svmsmote_edited)
print("svmsmote+edited quantification:",main_classification_svmsmote_edited.quantization_error)
print("svmsmote+edited topological:",main_classification_svmsmote_edited.topological_error)

main_classification_svmsmote_cnn= GEMA.Classification(main_map, data_svmsmote_cnn)
print("svmsmote+cnn quantification:",main_classification_svmsmote_cnn.quantization_error)
print("svmsmote+cnn topological:",main_classification_svmsmote_cnn.topological_error)

main_classification_svmsmote_ncr= GEMA.Classification(main_map, data_svmsmote_ncr)
print("svmsmote+ncr quantification:",main_classification_svmsmote_ncr.quantization_error)
print("svmsmote+ncr topological:",main_classification_svmsmote_ncr.topological_error)

main_classification_svmsmote_oss= GEMA.Classification(main_map, data_svmsmote_oss)
print("svmsmote+oss quantification:",main_classification_svmsmote_oss.quantization_error)
print("svmsmote+oss topological:",main_classification_svmsmote_oss.topological_error)

## Clasificación créditos Minisom y SVM SMOTE

In [ ]:
print("svmsmote+tommed")
print(som.quantization_error(data_svmsmote_tomek), som.topographic_error(data_svmsmote_tomek))
print("svmsmote+linked")
print(som.quantization_error(data_svmsmote_edited), som.topographic_error(data_svmsmote_edited))
print("svmsmote+cnn")
print(som.quantization_error(data_svmsmote_cnn), som.topographic_error(data_svmsmote_cnn))
print("svmsmote+ncr")
print(som.quantization_error(data_svmsmote_ncr), som.topographic_error(data_svmsmote_ncr))
print("svmsmote+oss")
print(som.quantization_error(data_svmsmote_oss), som.topographic_error(data_svmsmote_oss))

## Kmeans SMOTE créditos

In [ ]:
## KMSSMOTE + Tomed_links
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 20)
data_std = wf.min_max_normalization(datos_mode)

kmssmote_tomek=wf.KMSSMOTE_tomek(data_std,20)
print(kmssmote_tomek[20].value_counts())
data_kmssmote_tomek = kmssmote_tomek.drop(columns = [20])
data_kmssmote_tomek=data_kmssmote_tomek.to_numpy()

## KMSSMOTE + ENN
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 20)
data_std = wf.min_max_normalization(datos_mode)

kmssmote_edited=wf.KMSSMOTE_enn(data_std,20)
print(kmssmote_edited[20].value_counts())
data_kmssmote_edited = kmssmote_edited.drop(columns = [20])
data_kmssmote_edited=data_kmssmote_edited.to_numpy()

## KMSSMOTE con CNN (condensed nearest neighbors)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 20)
data_std = wf.min_max_normalization(datos_mode)

kmssmote_cnn=wf.KMSSMOTE_cnn(data_std,20)
print(kmssmote_cnn[20].value_counts())
data_kmssmote_cnn = kmssmote_cnn.drop(columns = [20])
data_kmssmote_cnn=data_kmssmote_cnn.to_numpy()

## KMSSMOTE con NCR (Neighboorhood cleaning rule)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 20)
data_std = wf.min_max_normalization(datos_mode)

kmssmote_ncr=wf.KMSSMOTE_ncr(data_std,20)
print(kmssmote_ncr[20].value_counts())
data_kmssmote_ncr = kmssmote_ncr.drop(columns = [20])
data_kmssmote_ncr=data_kmssmote_ncr.to_numpy()

## KMSSMOTE con OSD (One sided selection)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 20)
data_std = wf.min_max_normalization(datos_mode)

kmssmote_oss=wf.KMSSMOTE_oss(data_std,20)
print(kmssmote_oss[20].value_counts())
data_kmssmote_oss = kmssmote_oss.drop(columns = [20])
data_kmssmote_oss=data_kmssmote_oss.to_numpy()

## Clasificación créditos con GEMA y Kmeans SMOTE

In [ ]:
main_classification_kmssmote_tomek = GEMA.Classification(main_map, data_kmssmote_tomek)
print("svmsmote+tommed quantification:",main_classification_kmssmote_tomek.quantization_error)
print("svmsmote+tommed topological:",main_classification_kmssmote_tomek.topological_error)

main_classification_kmssmote_edited= GEMA.Classification(main_map, data_kmssmote_edited)
print("svmsmote+edited quantification:",main_classification_kmssmote_edited.quantization_error)
print("svmsmote+edited topological:",main_classification_kmssmote_edited.topological_error)

main_classification_kmssmote_cnn= GEMA.Classification(main_map, data_kmssmote_cnn)
print("svmsmote+cnn quantification:",main_classification_kmssmote_cnn.quantization_error)
print("svmsmote+cnn topological:",main_classification_kmssmote_cnn.topological_error)

main_classification_kmssmote_ncr= GEMA.Classification(main_map, data_kmssmote_ncr)
print("svmsmote+ncr quantification:",main_classification_kmssmote_ncr.quantization_error)
print("svmsmote+ncr topological:",main_classification_kmssmote_ncr.topological_error)

main_classification_kmssmote_oss= GEMA.Classification(main_map, data_kmssmote_oss)
print("svmsmote+oss quantification:",main_classification_kmssmote_oss.quantization_error)
print("svmsmote+oss topological:",main_classification_kmssmote_oss.topological_error)

## Clasificación créditos con Minisom y Kmeans SMOTE

In [ ]:
print("kmssmote+tommed")
print(som.quantization_error(data_kmssmote_tomek), som.topographic_error(data_kmssmote_tomek))
print("kmssmote+linked")
print(som.quantization_error(data_kmssmote_edited), som.topographic_error(data_kmssmote_edited))
print("kmssmote+cnn")
print(som.quantization_error(data_kmssmote_cnn), som.topographic_error(data_kmssmote_cnn))
print("kmssmote+ncr")
print(som.quantization_error(data_kmssmote_ncr), som.topographic_error(data_kmssmote_ncr))
print("kmssmote+oss")
print(som.quantization_error(data_kmssmote_oss), som.topographic_error(data_kmssmote_oss))

## Fonemas

In [51]:
## Importar dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/phoneme.csv'
missing_values = [' ', 'NaN', 'na', 'Na', '-', '--', 'n/a', '?']
columns = ['class']
data = pd.read_csv(url, na_values = missing_values, header=None)

In [ ]:
data

In [ ]:
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 5)
datos_mode[5].value_counts()

In [53]:
datos_mode=datos_mode.drop(columns = [5])
data_std = wf.min_max_normalization(datos_mode)
target = data[5].values
label_names={0:'Nasals', 1:'Orals'}
data_som=data_std.to_numpy()

## Entrenamiento GEMA fonemas

In [ ]:
main_map = GEMA.Map(data_som, 
                    19, 
                    1000, 
                    initial_lr=0.2, 
                    initial_neighbourhood = 0,
                    distance = 'euclidean',
                    use_decay= False,
                    presentation= 'random',
                    weights = 'random')

main_classification = GEMA.Classification(main_map, data_som)
print("quantification error:",main_classification.quantization_error)
print("Topological_error:",main_classification.topological_error)

In [ ]:
#import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.colors import LinearSegmentedColormap

porcentaje = 0.8

# Inicializa un mapa de colores del tamaño del SOM con todos los valores en blanco
color_map = np.ones((19, 19, 3))

# Inicializa un mapa de clases del tamaño del SOM con todos los valores en -1
class_map = -1 * np.ones((19, 19))

# Contadores para cada neurona
counts = np.zeros((19, 19, 2), dtype=int)

# Itera sobre cada dato y su clase correspondiente
for datum, class_ in zip(data_som, target):
    # Encuentra la neurona ganadora para este dato
    winner = main_map.calculate_bmu(datum)[1]
    
    # Actualiza el contador de la clase para la neurona ganadora
    counts[int(winner[0]), int(winner[1]), int(class_)] += 1
    
    # Actualiza el mapa de clases con la clase reconocida
    class_map[winner] = class_

# Itera sobre cada neurona para determinar el color final
for i in range(19):
    for j in range(19):
        # Calcula el porcentaje de cada clase para la neurona actual
        percent_class_0 = counts[i, j, 0] / (counts[i, j, 0] + counts[i, j, 1]) if counts[i, j, 0] + counts[i, j, 1] > 0 else 0
        percent_class_1 = counts[i, j, 1] / (counts[i, j, 0] + counts[i, j, 1]) if counts[i, j, 0] + counts[i, j, 1] > 0 else 0
        
        # Asigna el color según las condiciones especificadas
        if percent_class_0 >= porcentaje:
            color_map[i, j] = [0, 0, 1]  # Azul
        elif percent_class_1 >= porcentaje:
            color_map[i, j] = [1, 0, 0]  # Rojo
        elif counts[i, j, 0] + counts[i, j, 1] == 0:
            color_map[i, j] = [1, 1, 1]  # Blanco si no ha reconocido ningún patrón
        else:
            color_map[i, j] = [0, 1, 0]  # Verde si no alcanza el 80% de ninguna clase


# Count the number of neurons of each color
num_blue = np.sum(np.all(color_map == [0, 0, 1], axis=-1))
num_red = np.sum(np.all(color_map == [1, 0, 0], axis=-1))
num_white = np.sum(np.all(color_map == [1, 1, 1], axis=-1))
num_green = np.sum(np.all(color_map == [0, 1, 0], axis=-1))

# Crea una figura con diferentes tamaños para los subplots
fig = plt.figure(figsize=(8, 6))

# Crea una leyenda para el primer gráfico
legend_elements = [Patch(facecolor='blue', edgecolor='b', label='Clase 0'),
                   Patch(facecolor='red', edgecolor='r', label='Clase 1'),
                   Patch(facecolor='green', edgecolor='g', label='Ambas clases')]
ax1 = fig.add_subplot(121)  # 121 significa "1 fila, 2 columnas, primer gráfico"
# ax1.legend(handles=legend_elements, loc='upper right')
ax1.imshow(np.rot90(color_map))
ax1.set_title('Mapa de neuronas w/o escala')

# Inicializa un mapa de neuronas con ceros
neuron_map = np.zeros((19, 19))
ganadores = np.zeros((19, 19))

# Itera sobre los datos y las clases
for datum, clss in zip(data_som, target):
    # Obtiene la neurona ganadora para el dato actual
    winner = main_map.calculate_bmu(datum)[1]
    ganadores[winner] = 1
    print(winner, clss)
    # Si la clase es 0, resta 1, si la clase es 1, suma 1
    neuron_map[winner[0], winner[1]] += 1 if clss == 1 else -1

# Normaliza el mapa de neuronas para que los valores estén entre -1 y 1
# neuron_map = neuron_map / np.max(np.abs(neuron_map))

# Define los colores de la escala como azul, verde y amarillo
cmap_colors = ["indigo", "royalblue", "limegreen", "yellow"]

# Crea el mapa de colores
cmap = LinearSegmentedColormap.from_list("mycmap", cmap_colors)

# Dibuja el mapa de calor
ax2 = fig.add_subplot(122)  # 122 significa "1 fila, 2 columnas, segundo gráfico"
im = ax2.imshow(np.rot90(neuron_map), cmap=cmap, interpolation='nearest')
ax2.set_title('Mapa de neuronas con escala')

# Muestra la barra de colores
fig.colorbar(im, ax=ax2)

# Ajusta el layout
plt.tight_layout()

# Muestra el gráfico
plt.show()

# Print the number of neurons of each color
print(f"Number of blue neurons: {num_blue}")
print(f"Number of red neurons: {num_red}")
print(f"Number of white neurons: {num_white}")
print(f"Number of green neurons: {num_green}")

In [ ]:
#GEMA.Visualization.codebook_vectors(main_map, np.array(["Not Spiled", "Spiled"]))

In [ ]:
n_neurons = 6
m_neurons = 6

som = MiniSom(n_neurons, m_neurons, data_som.shape[1], learning_rate=0.12947086423703813, random_seed=42)
som.train(data_som, num_iteration=4900, verbose=True)  # random training
print("Topological error:",som.topographic_error(data_som))

## SMOTE fonemas

In [ ]:
## Smote + Tomed_links
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 5)
data_std = wf.min_max_normalization(datos_mode)

Smote_tomed=wf.smote_tomed_link(data_std,5)
print(Smote_tomed[5].value_counts())
data_balanced_dropped_smote_tomed = Smote_tomed.drop(columns = [5])
data_balanced_dropped_smote_tomed=data_balanced_dropped_smote_tomed.to_numpy()

## smote_edited_nearest_neighbor
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 5)
data_std = wf.min_max_normalization(datos_mode)

Smote_edited=wf.smote_edited_nearest_neighbor(data_std,5)
print(Smote_edited[5].value_counts())
data_balanced_dropped_smote_edited = Smote_edited.drop(columns = [5])
data_balanced_dropped_smote_edited=data_balanced_dropped_smote_edited.to_numpy()

## smote con CNN (condensed nearest neighbors)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 5)
data_std = wf.min_max_normalization(datos_mode)

Smote_cnn=wf.smote_cnn(data_std,5)
print(Smote_cnn[5].value_counts())
data_smote_cnn = Smote_cnn.drop(columns = [5])
data_smote_cnn=data_smote_cnn.to_numpy()

## smote con NCR (Neighboorhood cleaning rule)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 5)
data_std = wf.min_max_normalization(datos_mode)

Smote_ncr=wf.smote_ncr(data_std,5)
print(Smote_ncr[5].value_counts())
data_smote_ncr = Smote_ncr.drop(columns = [5])
data_smote_ncr=data_smote_ncr.to_numpy()

## smote con OSD (One sided selection)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 5)
data_std = wf.min_max_normalization(datos_mode)

Smote_oss=wf.smote_osd(data_std,5)
print(Smote_oss[5].value_counts())
data_smote_oss = Smote_oss.drop(columns = [5])
data_smote_oss=data_smote_oss.to_numpy()

## Clasificación fonemas con GEMA y SMOTE

In [ ]:
main_classification_smoted_tomed = GEMA.Classification(main_map, data_balanced_dropped_smote_tomed)
print("Smote+tommed quantification:",main_classification_smoted_tomed.quantization_error)
print("Smote+tommed topological:",main_classification_smoted_tomed.topological_error)

main_classification_smoted_edited= GEMA.Classification(main_map, data_balanced_dropped_smote_edited)
print("Smote+edited quantification:",main_classification_smoted_edited.quantization_error)
print("Smote+edited topological:",main_classification_smoted_edited.topological_error)

main_classification_smoted_cnn= GEMA.Classification(main_map, data_smote_cnn)
print("Smote+cnn quantification:",main_classification_smoted_cnn.quantization_error)
print("Smote+cnn topological:",main_classification_smoted_cnn.topological_error)

main_classification_smoted_ncr= GEMA.Classification(main_map, data_smote_ncr)
print("Smote+ncr quantification:",main_classification_smoted_ncr.quantization_error)
print("Smote+ncr topological:",main_classification_smoted_ncr.topological_error)

main_classification_smoted_oss= GEMA.Classification(main_map, data_smote_oss)
print("Smote+oss quantification:",main_classification_smoted_oss.quantization_error)
print("Smote+oss topological:",main_classification_smoted_oss.topological_error)

## Clasificación fonemas con Minisom y SMOTE

In [ ]:
print("Smote+tommed")
print(som.quantization_error(data_balanced_dropped_smote_tomed), som.topographic_error(data_balanced_dropped_smote_tomed))
print("Smote+linked")
print(som.quantization_error(data_balanced_dropped_smote_edited), som.topographic_error(data_balanced_dropped_smote_edited))
print("Smote+cnn")
print(som.quantization_error(data_smote_cnn), som.topographic_error(data_smote_cnn))
print("Smote+ncr")
print(som.quantization_error(data_smote_ncr), som.topographic_error(data_smote_ncr))
print("Smote+oss")
print(som.quantization_error(data_smote_oss), som.topographic_error(data_smote_oss))

## ADASYN fonemas

In [ ]:
## ADASYN + Tomed_links
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 5)
data_std = wf.min_max_normalization(datos_mode)

adasyn_tomek=wf.adasyn_tomek(data_std,5)
print(adasyn_tomek[5].value_counts())
data_adasyn_tomek = adasyn_tomek.drop(columns = [5])
data_adasyn_tomek=data_adasyn_tomek.to_numpy()

## ADASYN + ENN
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 5)
data_std = wf.min_max_normalization(datos_mode)

adasyn_edited=wf.adasyn_enn(data_std,5)
print(adasyn_edited[5].value_counts())
data_adasyn_edited = adasyn_edited.drop(columns = [5])
data_adasyn_edited=data_adasyn_edited.to_numpy()

## ADASYN con CNN (condensed nearest neighbors)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 5)
data_std = wf.min_max_normalization(datos_mode)

adasyn_cnn=wf.adasyn_cnn_fon(data_std,5)
print(adasyn_cnn[5].value_counts())
data_adasyn_cnn = adasyn_cnn.drop(columns = [5])
data_adasyn_cnn=data_adasyn_cnn.to_numpy()

## ADASYN con NCR (Neighboorhood cleaning rule)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 5)
data_std = wf.min_max_normalization(datos_mode)

adasyn_ncr=wf.adasyn_ncr(data_std,5)
print(adasyn_ncr[5].value_counts())
data_adasyn_ncr = adasyn_ncr.drop(columns = [5])
data_adasyn_ncr=data_adasyn_ncr.to_numpy()

## ADASYN con OSD (One sided selection)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 5)
data_std = wf.min_max_normalization(datos_mode)

adasyn_oss=wf.adasyn_oss(data_std,5)
print(adasyn_oss[5].value_counts())
data_adasyn_oss = adasyn_oss.drop(columns = [5])
data_adasyn_oss=data_adasyn_oss.to_numpy()

## Clasificación fonemas con GEMA y ADASYN

In [ ]:
main_classification_adasyn_tomek = GEMA.Classification(main_map, data_adasyn_tomek)
print("adasyn+tommed quantification:",main_classification_adasyn_tomek.quantization_error)
print("adasyn+tommed topological:",main_classification_adasyn_tomek.topological_error)

main_classification_adasyn_edited= GEMA.Classification(main_map, data_adasyn_edited)
print("adasyn+edited quantification:",main_classification_adasyn_edited.quantization_error)
print("adasyn+edited topological:",main_classification_adasyn_edited.topological_error)

main_classification_smoted_cnn= GEMA.Classification(main_map, data_smote_cnn)
print("adasyn+cnn quantification:",main_classification_smoted_cnn.quantization_error)
print("adasyn+cnn topological:",main_classification_smoted_cnn.topological_error)

main_classification_smoted_ncr= GEMA.Classification(main_map, data_smote_ncr)
print("adasyn+ncr quantification:",main_classification_smoted_ncr.quantization_error)
print("adasyn+ncr topological:",main_classification_smoted_ncr.topological_error)

main_classification_smoted_oss= GEMA.Classification(main_map, data_smote_oss)
print("adasyn+oss quantification:",main_classification_smoted_oss.quantization_error)
print("adasyn+oss topological:",main_classification_smoted_oss.topological_error)

## Clasificación fonemas con Minisom y ADASYN

In [ ]:
print("adasyn+tommed")
print(som.quantization_error(data_adasyn_tomek), som.topographic_error(data_adasyn_tomek))
print("adasyn+linked")
print(som.quantization_error(data_adasyn_edited), som.topographic_error(data_adasyn_edited))
print("adasyn+cnn")
print(som.quantization_error(data_adasyn_cnn), som.topographic_error(data_adasyn_cnn))
print("adasyn+ncr")
print(som.quantization_error(data_adasyn_ncr), som.topographic_error(data_adasyn_ncr))
print("adasyn+oss")
print(som.quantization_error(data_adasyn_oss), som.topographic_error(data_adasyn_oss))

## Borderline SMOTE fonemas

In [ ]:
## BSMOTE + Tomed_links
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 5)
data_std = wf.min_max_normalization(datos_mode)

bsmote_tomek=wf.BSMOTE_tomek(data_std,5)
print(bsmote_tomek[5].value_counts())
data_bsmote_tomek = bsmote_tomek.drop(columns = [5])
data_bsmote_tomek=data_bsmote_tomek.to_numpy()

## BSMOTE + ENN
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 5)
data_std = wf.min_max_normalization(datos_mode)

bsmote_edited=wf.BSMOTE_enn(data_std,5)
print(bsmote_edited[5].value_counts())
data_bsmote_edited = bsmote_edited.drop(columns = [5])
data_bsmote_edited=data_bsmote_edited.to_numpy()

## BSMOTE con CNN (condensed nearest neighbors)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 5)
data_std = wf.min_max_normalization(datos_mode)

bsmote_cnn=wf.BSMOTE_cnn(data_std,5)
print(bsmote_cnn[5].value_counts())
data_bsmote_cnn = bsmote_cnn.drop(columns = [5])
data_bsmote_cnn=data_bsmote_cnn.to_numpy()

## BSMOTE con NCR (Neighboorhood cleaning rule)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 5)
data_std = wf.min_max_normalization(datos_mode)

bsmote_ncr=wf.BSMOTE_ncr(data_std,5)
print(bsmote_ncr[5].value_counts())
data_bsmote_ncr = bsmote_ncr.drop(columns = [5])
data_bsmote_ncr=data_bsmote_ncr.to_numpy()

## BSMOTE con OSD (One sided selection)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 5)
data_std = wf.min_max_normalization(datos_mode)

bsmote_oss=wf.BSMOTE_oss(data_std,5)
print(bsmote_oss[5].value_counts())
data_bsmote_oss = bsmote_oss.drop(columns = [5])
data_bsmote_oss=data_bsmote_oss.to_numpy()

## Clasificación fonemas con GEMA y Borderline SMOTE

In [ ]:
main_classification_bsmote_tomek = GEMA.Classification(main_map, data_bsmote_tomek)
print("bsmote+tommed quantification:",main_classification_bsmote_tomek.quantization_error)
print("bsmote+tommed topological:",main_classification_bsmote_tomek.topological_error)

main_classification_bsmote_edited= GEMA.Classification(main_map, data_bsmote_edited)
print("bsmote+edited quantification:",main_classification_bsmote_edited.quantization_error)
print("bsmote+edited topological:",main_classification_bsmote_edited.topological_error)

main_classification_bsmote_cnn= GEMA.Classification(main_map, data_bsmote_cnn)
print("bsmote+cnn quantification:",main_classification_bsmote_cnn.quantization_error)
print("bsmote+cnn topological:",main_classification_bsmote_cnn.topological_error)

main_classification_bsmote_ncr= GEMA.Classification(main_map, data_bsmote_ncr)
print("bsmote+ncr quantification:",main_classification_bsmote_ncr.quantization_error)
print("bsmote+ncr topological:",main_classification_bsmote_ncr.topological_error)

main_classification_bsmote_oss= GEMA.Classification(main_map, data_bsmote_oss)
print("bsmote+oss quantification:",main_classification_bsmote_oss.quantization_error)
print("bsmote+oss topological:",main_classification_bsmote_oss.topological_error)

## Clasificación fonemas con Minisom y Borderline SMOTE

In [ ]:
print("bsmote+tommed")
print(som.quantization_error(data_bsmote_tomek), som.topographic_error(data_bsmote_tomek))
print("bsmote+linked")
print(som.quantization_error(data_bsmote_edited), som.topographic_error(data_bsmote_edited))
print("bsmote+cnn")
print(som.quantization_error(data_bsmote_cnn), som.topographic_error(data_bsmote_cnn))
print("bsmote+ncr")
print(som.quantization_error(data_bsmote_ncr), som.topographic_error(data_bsmote_ncr))
print("bsmote+oss")
print(som.quantization_error(data_bsmote_oss), som.topographic_error(data_bsmote_oss))

## SVM SMOTE fonemas

In [ ]:
## SVMSMOTE + Tomed_links
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 5)
data_std = wf.min_max_normalization(datos_mode)

svmsmote_tomek=wf.SVMSMOTE_tomek(data_std,5)
print(svmsmote_tomek[5].value_counts())
data_svmsmote_tomek = svmsmote_tomek.drop(columns = [5])
data_svmsmote_tomek=data_svmsmote_tomek.to_numpy()

## SVMSMOTE + ENN
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 5)
data_std = wf.min_max_normalization(datos_mode)

svmsmote_edited=wf.SVMSMOTE_enn(data_std,5)
print(svmsmote_edited[5].value_counts())
data_svmsmote_edited = svmsmote_edited.drop(columns = [5])
data_svmsmote_edited=data_svmsmote_edited.to_numpy()

## SVMSMOTE con CNN (condensed nearest neighbors)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 5)
data_std = wf.min_max_normalization(datos_mode)

svmsmote_cnn=wf.SVMSMOTE_cnn(data_std,5)
print(svmsmote_cnn[5].value_counts())
data_svmsmote_cnn = svmsmote_cnn.drop(columns = [5])
data_svmsmote_cnn=data_svmsmote_cnn.to_numpy()

## SVMSMOTE con NCR (Neighboorhood cleaning rule)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 5)
data_std = wf.min_max_normalization(datos_mode)

svmsmote_ncr=wf.SVMSMOTE_ncr(data_std,5)
print(svmsmote_ncr[5].value_counts())
data_svmsmote_ncr = svmsmote_ncr.drop(columns = [5])
data_svmsmote_ncr=data_svmsmote_ncr.to_numpy()

## SVMSMOTE con OSD (One sided selection)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 5)
data_std = wf.min_max_normalization(datos_mode)

svmsmote_oss=wf.SVMSMOTE_oss(data_std,5)
print(svmsmote_oss[5].value_counts())
data_svmsmote_oss = svmsmote_oss.drop(columns = [5])
data_svmsmote_oss=data_svmsmote_oss.to_numpy()

## Clasificación fonemas con GEMA y SVM SMOTE

In [ ]:
main_classification_svmsmote_tomek = GEMA.Classification(main_map, data_svmsmote_tomek)
print("svmsmote+tommed quantification:",main_classification_svmsmote_tomek.quantization_error)
print("svmsmote+tommed topological:",main_classification_svmsmote_tomek.topological_error)

main_classification_svmsmote_edited= GEMA.Classification(main_map, data_svmsmote_edited)
print("svmsmote+edited quantification:",main_classification_svmsmote_edited.quantization_error)
print("svmsmote+edited topological:",main_classification_svmsmote_edited.topological_error)

main_classification_svmsmote_cnn= GEMA.Classification(main_map, data_svmsmote_cnn)
print("svmsmote+cnn quantification:",main_classification_svmsmote_cnn.quantization_error)
print("svmsmote+cnn topological:",main_classification_svmsmote_cnn.topological_error)

main_classification_svmsmote_ncr= GEMA.Classification(main_map, data_svmsmote_ncr)
print("svmsmote+ncr quantification:",main_classification_svmsmote_ncr.quantization_error)
print("svmsmote+ncr topological:",main_classification_svmsmote_ncr.topological_error)

main_classification_svmsmote_oss= GEMA.Classification(main_map, data_svmsmote_oss)
print("svmsmote+oss quantification:",main_classification_svmsmote_oss.quantization_error)
print("svmsmote+oss topological:",main_classification_svmsmote_oss.topological_error)

## Clasificación fonemas con Minisom y SVM SMOTE

In [ ]:
print("svmsmote+tommed")
print(som.quantization_error(data_svmsmote_tomek), som.topographic_error(data_svmsmote_tomek))
print("svmsmote+linked")
print(som.quantization_error(data_svmsmote_edited), som.topographic_error(data_svmsmote_edited))
print("svmsmote+cnn")
print(som.quantization_error(data_svmsmote_cnn), som.topographic_error(data_svmsmote_cnn))
print("svmsmote+ncr")
print(som.quantization_error(data_svmsmote_ncr), som.topographic_error(data_svmsmote_ncr))
print("svmsmote+oss")
print(som.quantization_error(data_svmsmote_oss), som.topographic_error(data_svmsmote_oss))

## Kmeans SMOTE fonemas

In [ ]:
## KMSSMOTE + Tomed_links
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 5)
data_std = wf.min_max_normalization(datos_mode)

kmssmote_tomek=wf.KMSSMOTE_tomek(data_std,5)
print(kmssmote_tomek[5].value_counts())
data_kmssmote_tomek = kmssmote_tomek.drop(columns = [5])
data_kmssmote_tomek=data_kmssmote_tomek.to_numpy()

## KMSSMOTE + ENN
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 5)
data_std = wf.min_max_normalization(datos_mode)

kmssmote_edited=wf.KMSSMOTE_enn(data_std,5)
print(kmssmote_edited[5].value_counts())
data_kmssmote_edited = kmssmote_edited.drop(columns = [5])
data_kmssmote_edited=data_kmssmote_edited.to_numpy()

## KMSSMOTE con CNN (condensed nearest neighbors)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 5)
data_std = wf.min_max_normalization(datos_mode)

kmssmote_cnn=wf.KMSSMOTE_cnn(data_std,5)
print(kmssmote_cnn[5].value_counts())
data_kmssmote_cnn = kmssmote_cnn.drop(columns = [5])
data_kmssmote_cnn=data_kmssmote_cnn.to_numpy()

## KMSSMOTE con NCR (Neighboorhood cleaning rule)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 5)
data_std = wf.min_max_normalization(datos_mode)

kmssmote_ncr=wf.KMSSMOTE_ncr(data_std,5)
print(kmssmote_ncr[5].value_counts())
data_kmssmote_ncr = kmssmote_ncr.drop(columns = [5])
data_kmssmote_ncr=data_kmssmote_ncr.to_numpy()

## KMSSMOTE con OSD (One sided selection)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 5)
data_std = wf.min_max_normalization(datos_mode)

kmssmote_oss=wf.KMSSMOTE_oss(data_std,5)
print(kmssmote_oss[5].value_counts())
data_kmssmote_oss = kmssmote_oss.drop(columns = [5])
data_kmssmote_oss=data_kmssmote_oss.to_numpy()

## Clasificación fonemas con GEMA y Kmeans SMOTE

In [ ]:
main_classification_kmssmote_tomek = GEMA.Classification(main_map, data_kmssmote_tomek)
print("svmsmote+tommed quantification:",main_classification_kmssmote_tomek.quantization_error)
print("svmsmote+tommed topological:",main_classification_kmssmote_tomek.topological_error)

main_classification_kmssmote_edited= GEMA.Classification(main_map, data_kmssmote_edited)
print("svmsmote+edited quantification:",main_classification_kmssmote_edited.quantization_error)
print("svmsmote+edited topological:",main_classification_kmssmote_edited.topological_error)

main_classification_kmssmote_cnn= GEMA.Classification(main_map, data_kmssmote_cnn)
print("svmsmote+cnn quantification:",main_classification_kmssmote_cnn.quantization_error)
print("svmsmote+cnn topological:",main_classification_kmssmote_cnn.topological_error)

main_classification_kmssmote_ncr= GEMA.Classification(main_map, data_kmssmote_ncr)
print("svmsmote+ncr quantification:",main_classification_kmssmote_ncr.quantization_error)
print("svmsmote+ncr topological:",main_classification_kmssmote_ncr.topological_error)

main_classification_kmssmote_oss= GEMA.Classification(main_map, data_kmssmote_oss)
print("svmsmote+oss quantification:",main_classification_kmssmote_oss.quantization_error)
print("svmsmote+oss topological:",main_classification_kmssmote_oss.topological_error)

## Clasificación fonemas con Minisom y Kmeans SMOTE

In [ ]:
print("kmssmote+tommed")
print(som.quantization_error(data_kmssmote_tomek), som.topographic_error(data_kmssmote_tomek))
print("kmssmote+linked")
print(som.quantization_error(data_kmssmote_edited), som.topographic_error(data_kmssmote_edited))
print("kmssmote+cnn")
print(som.quantization_error(data_kmssmote_cnn), som.topographic_error(data_kmssmote_cnn))
print("kmssmote+ncr")
print(som.quantization_error(data_kmssmote_ncr), som.topographic_error(data_kmssmote_ncr))
print("kmssmote+oss")
print(som.quantization_error(data_kmssmote_oss), som.topographic_error(data_kmssmote_oss))

## Importar dataset de fraudes de créditos bancarios

In [ ]:
## Importar dataset
url = '../../PROYECTO SOM DESBALANCEADO/DATASET/creditcard.csv'
missing_values = [' ', 'NaN', 'na', 'Na', '-', '--', 'n/a', '?']
columns = ['class']
data = pd.read_csv(url, na_values = missing_values, header=None)
data[3] = data[3].replace({1: 0, 2: 1})
print(data[30].value_counts())

In [ ]:
data = data.iloc[1:]
data

In [ ]:
data = data.astype(float)
# Contar cuántas filas tienen un valor de 0 en la columna "columna_nombre"
filas_con_cero = data[data[30] == 0]

# Verificar si hay suficientes filas con 0 antes de eliminar
if len(filas_con_cero) >= 274000:
    # Eliminar las primeras 180,000 filas con 0
    data = data.drop(filas_con_cero.index[:274000])
datos_mode = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(datos_mode, 30)
datos_mode[30].value_counts()

In [65]:
datos_mode=datos_mode.drop(columns = [30])
data_std = wf.min_max_normalization(datos_mode)
target = data[30].values
label_names={0:'No fraude', 1:'Fraude'}
data_som=data_std.to_numpy()

## Entrenamiento GEMA fraudes créditos

In [ ]:
main_map = GEMA.Map(data_som, 
                    20, 
                    38386, 
                    initial_lr=0.01819940721573887, 
                    initial_neighbourhood = 0,
                    distance = 'euclidean',
                    use_decay= False,
                    presentation= 'random',
                    weights = 'random')

main_classification = GEMA.Classification(main_map, data_som)
print("quantification error:",main_classification.quantization_error)
print("Topological_error:",main_classification.topological_error)

## Entrenamiento SOM Minisom fraudes créditos

In [ ]:
n_neurons = 20
m_neurons = 20

som = MiniSom(n_neurons, m_neurons, data_som.shape[1], learning_rate=0.005, random_seed=42)
som.train(data_som, num_iteration=38386, verbose=True)  # random training
print("Topological error:",som.topographic_error(data_som))

## SMOTE fraudes créditos

In [ ]:
## Smote + Tomed_links
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 30)
data_std = wf.min_max_normalization(datos_mode)

Smote_tomed=wf.smote_tomed_link(data_std,30)
print(Smote_tomed[30].value_counts())
data_balanced_dropped_smote_tomed = Smote_tomed.drop(columns = [30])
data_balanced_dropped_smote_tomed=data_balanced_dropped_smote_tomed.to_numpy()

## smote_edited_nearest_neighbor
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 30)
data_std = wf.min_max_normalization(datos_mode)

Smote_edited=wf.smote_edited_nearest_neighbor(data_std,30)
print(Smote_edited[30].value_counts())
data_balanced_dropped_smote_edited = Smote_edited.drop(columns = [30])
data_balanced_dropped_smote_edited=data_balanced_dropped_smote_edited.to_numpy()

## smote con CNN (condensed nearest neighbors)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 30)
data_std = wf.min_max_normalization(datos_mode)

Smote_cnn=wf.smote_cnn(data_std,30)
print(Smote_cnn[30].value_counts())
data_smote_cnn = Smote_cnn.drop(columns = [30])
data_smote_cnn=data_smote_cnn.to_numpy()

## smote con NCR (Neighboorhood cleaning rule)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 30)
data_std = wf.min_max_normalization(datos_mode)

Smote_ncr=wf.smote_ncr(data_std,30)
print(Smote_ncr[30].value_counts())
data_smote_ncr = Smote_ncr.drop(columns = [30])
data_smote_ncr=data_smote_ncr.to_numpy()

## smote con OSD (One sided selection)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 30)
data_std = wf.min_max_normalization(datos_mode)

Smote_oss=wf.smote_osd(data_std,30)
print(Smote_oss[30].value_counts())
data_smote_oss = Smote_oss.drop(columns = [30])
data_smote_oss=data_smote_oss.to_numpy()

## Clasificación GEMA SMOTE fraudes

In [ ]:
main_classification_smoted_tomed = GEMA.Classification(main_map, data_balanced_dropped_smote_tomed)
print("Smote+tommed quantification:",main_classification_smoted_tomed.quantization_error)
print("Smote+tommed topological:",main_classification_smoted_tomed.topological_error)

main_classification_smoted_edited= GEMA.Classification(main_map, data_balanced_dropped_smote_edited)
print("Smote+edited quantification:",main_classification_smoted_edited.quantization_error)
print("Smote+edited topological:",main_classification_smoted_edited.topological_error)

main_classification_smoted_cnn= GEMA.Classification(main_map, data_smote_cnn)
print("Smote+cnn quantification:",main_classification_smoted_cnn.quantization_error)
print("Smote+cnn topological:",main_classification_smoted_cnn.topological_error)

main_classification_smoted_ncr= GEMA.Classification(main_map, data_smote_ncr)
print("Smote+ncr quantification:",main_classification_smoted_ncr.quantization_error)
print("Smote+ncr topological:",main_classification_smoted_ncr.topological_error)

main_classification_smoted_oss= GEMA.Classification(main_map, data_smote_oss)
print("Smote+oss quantification:",main_classification_smoted_oss.quantization_error)
print("Smote+oss topological:",main_classification_smoted_oss.topological_error)

## Clasificación Minisom SMOTE fraudes

In [ ]:
print("Smote+tommed")
print(som.quantization_error(data_balanced_dropped_smote_tomed), som.topographic_error(data_balanced_dropped_smote_tomed))
print("Smote+linked")
print(som.quantization_error(data_balanced_dropped_smote_edited), som.topographic_error(data_balanced_dropped_smote_edited))
print("Smote+cnn")
print(som.quantization_error(data_smote_cnn), som.topographic_error(data_smote_cnn))
print("Smote+ncr")
print(som.quantization_error(data_smote_ncr), som.topographic_error(data_smote_ncr))
print("Smote+oss")
print(som.quantization_error(data_smote_oss), som.topographic_error(data_smote_oss))

## ADASYN fraudes

In [ ]:
## ADASYN + Tomed_links
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 30)
data_std = wf.min_max_normalization(datos_mode)

adasyn_tomek=wf.adasyn_tomek(data_std,30)
print(adasyn_tomek[30].value_counts())
data_adasyn_tomek = adasyn_tomek.drop(columns = [30])
data_adasyn_tomek=data_adasyn_tomek.to_numpy()

## ADASYN + ENN
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 30)
data_std = wf.min_max_normalization(datos_mode)

adasyn_edited=wf.adasyn_enn(data_std,30)
print(adasyn_edited[30].value_counts())
data_adasyn_edited = adasyn_edited.drop(columns = [30])
data_adasyn_edited=data_adasyn_edited.to_numpy()

## ADASYN con CNN (condensed nearest neighbors)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 30)
data_std = wf.min_max_normalization(datos_mode)

adasyn_cnn=wf.adasyn_cnn_fon(data_std,30)
print(adasyn_cnn[30].value_counts())
data_adasyn_cnn = adasyn_cnn.drop(columns = [30])
data_adasyn_cnn=data_adasyn_cnn.to_numpy()

## ADASYN con NCR (Neighboorhood cleaning rule)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 30)
data_std = wf.min_max_normalization(datos_mode)

adasyn_ncr=wf.adasyn_ncr(data_std,30)
print(adasyn_ncr[30].value_counts())
data_adasyn_ncr = adasyn_ncr.drop(columns = [30])
data_adasyn_ncr=data_adasyn_ncr.to_numpy()

## ADASYN con OSD (One sided selection)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 30)
data_std = wf.min_max_normalization(datos_mode)

adasyn_oss=wf.adasyn_oss(data_std,30)
print(adasyn_oss[30].value_counts())
data_adasyn_oss = adasyn_oss.drop(columns = [30])
data_adasyn_oss=data_adasyn_oss.to_numpy()

## Clasificación GEMA ADASYN fraudes

In [ ]:
main_classification_adasyn_tomek = GEMA.Classification(main_map, data_adasyn_tomek)
print("adasyn+tommed quantification:",main_classification_adasyn_tomek.quantization_error)
print("adasyn+tommed topological:",main_classification_adasyn_tomek.topological_error)

main_classification_adasyn_edited= GEMA.Classification(main_map, data_adasyn_edited)
print("adasyn+edited quantification:",main_classification_adasyn_edited.quantization_error)
print("adasyn+edited topological:",main_classification_adasyn_edited.topological_error)

main_classification_smoted_cnn= GEMA.Classification(main_map, data_adasyn_cnn)
print("adasyn+cnn quantification:",main_classification_smoted_cnn.quantization_error)
print("adasyn+cnn topological:",main_classification_smoted_cnn.topological_error)

main_classification_smoted_ncr= GEMA.Classification(main_map, data_adasyn_ncr)
print("adasyn+ncr quantification:",main_classification_smoted_ncr.quantization_error)
print("adasyn+ncr topological:",main_classification_smoted_ncr.topological_error)

main_classification_smoted_oss= GEMA.Classification(main_map, data_smote_oss)
print("adasyn+oss quantification:",main_classification_smoted_oss.quantization_error)
print("adasyn+oss topological:",main_classification_smoted_oss.topological_error)

## Clasificación Minisom ADASYN fraudes

In [ ]:
print("adasyn+tommed")
print(som.quantization_error(data_adasyn_tomek), som.topographic_error(data_adasyn_tomek))
print("adasyn+linked")
print(som.quantization_error(data_adasyn_edited), som.topographic_error(data_adasyn_edited))
print("adasyn+cnn")
print(som.quantization_error(data_adasyn_cnn), som.topographic_error(data_adasyn_cnn))
print("adasyn+ncr")
print(som.quantization_error(data_adasyn_ncr), som.topographic_error(data_adasyn_ncr))
print("adasyn+oss")
print(som.quantization_error(data_adasyn_oss), som.topographic_error(data_adasyn_oss))

## Borderline SMOTE fraudes

In [ ]:
## BSMOTE + Tomed_links
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 30)
data_std = wf.min_max_normalization(datos_mode)

bsmote_tomek=wf.BSMOTE_tomek(data_std,30)
print(bsmote_tomek[30].value_counts())
data_bsmote_tomek = bsmote_tomek.drop(columns = [30])
data_bsmote_tomek=data_bsmote_tomek.to_numpy()

## BSMOTE + ENN
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 30)
data_std = wf.min_max_normalization(datos_mode)

bsmote_edited=wf.BSMOTE_enn(data_std,30)
print(bsmote_edited[30].value_counts())
data_bsmote_edited = bsmote_edited.drop(columns = [30])
data_bsmote_edited=data_bsmote_edited.to_numpy()

## BSMOTE con CNN (condensed nearest neighbors)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 30)
data_std = wf.min_max_normalization(datos_mode)

bsmote_cnn=wf.BSMOTE_cnn(data_std,30)
print(bsmote_cnn[30].value_counts())
data_bsmote_cnn = bsmote_cnn.drop(columns = [30])
data_bsmote_cnn=data_bsmote_cnn.to_numpy()

## BSMOTE con NCR (Neighboorhood cleaning rule)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 30)
data_std = wf.min_max_normalization(datos_mode)

bsmote_ncr=wf.BSMOTE_ncr(data_std,30)
print(bsmote_ncr[30].value_counts())
data_bsmote_ncr = bsmote_ncr.drop(columns = [30])
data_bsmote_ncr=data_bsmote_ncr.to_numpy()

## BSMOTE con OSD (One sided selection)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 30)
data_std = wf.min_max_normalization(datos_mode)

bsmote_oss=wf.BSMOTE_oss(data_std,30)
print(bsmote_oss[30].value_counts())
data_bsmote_oss = bsmote_oss.drop(columns = [30])
data_bsmote_oss=data_bsmote_oss.to_numpy()

## Clasificación GEMA BSMOTE Fraudes

In [ ]:
main_classification_bsmote_tomek = GEMA.Classification(main_map, data_bsmote_tomek)
print("bsmote+tommed quantification:",main_classification_bsmote_tomek.quantization_error)
print("bsmote+tommed topological:",main_classification_bsmote_tomek.topological_error)

main_classification_bsmote_edited= GEMA.Classification(main_map, data_bsmote_edited)
print("bsmote+edited quantification:",main_classification_bsmote_edited.quantization_error)
print("bsmote+edited topological:",main_classification_bsmote_edited.topological_error)

main_classification_bsmote_cnn= GEMA.Classification(main_map, data_bsmote_cnn)
print("bsmote+cnn quantification:",main_classification_bsmote_cnn.quantization_error)
print("bsmote+cnn topological:",main_classification_bsmote_cnn.topological_error)

main_classification_bsmote_ncr= GEMA.Classification(main_map, data_bsmote_ncr)
print("bsmote+ncr quantification:",main_classification_bsmote_ncr.quantization_error)
print("bsmote+ncr topological:",main_classification_bsmote_ncr.topological_error)

main_classification_bsmote_oss= GEMA.Classification(main_map, data_bsmote_oss)
print("bsmote+oss quantification:",main_classification_bsmote_oss.quantization_error)
print("bsmote+oss topological:",main_classification_bsmote_oss.topological_error)

## Clasificación Minisom BSMOTE Fraudes

In [ ]:
print("bsmote+tommed")
print(som.quantization_error(data_bsmote_tomek), som.topographic_error(data_bsmote_tomek))
print("bsmote+linked")
print(som.quantization_error(data_bsmote_edited), som.topographic_error(data_bsmote_edited))
print("bsmote+cnn")
print(som.quantization_error(data_bsmote_cnn), som.topographic_error(data_bsmote_cnn))
print("bsmote+ncr")
print(som.quantization_error(data_bsmote_ncr), som.topographic_error(data_bsmote_ncr))
print("bsmote+oss")
print(som.quantization_error(data_bsmote_oss), som.topographic_error(data_bsmote_oss))

## SVM SMOTE fraudes

In [ ]:
## SVMSMOTE + Tomed_links
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 30)
data_std = wf.min_max_normalization(datos_mode)

svmsmote_tomek=wf.SVMSMOTE_tomek(data_std,30)
print(svmsmote_tomek[30].value_counts())
data_svmsmote_tomek = svmsmote_tomek.drop(columns = [30])
data_svmsmote_tomek=data_svmsmote_tomek.to_numpy()

## SVMSMOTE + ENN
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 30)
data_std = wf.min_max_normalization(datos_mode)

svmsmote_edited=wf.SVMSMOTE_enn(data_std,30)
print(svmsmote_edited[30].value_counts())
data_svmsmote_edited = svmsmote_edited.drop(columns = [30])
data_svmsmote_edited=data_svmsmote_edited.to_numpy()

## SVMSMOTE con CNN (condensed nearest neighbors)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 30)
data_std = wf.min_max_normalization(datos_mode)

svmsmote_cnn=wf.SVMSMOTE_cnn(data_std,30)
print(svmsmote_cnn[30].value_counts())
data_svmsmote_cnn = svmsmote_cnn.drop(columns = [30])
data_svmsmote_cnn=data_svmsmote_cnn.to_numpy()

## SVMSMOTE con NCR (Neighboorhood cleaning rule)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 30)
data_std = wf.min_max_normalization(datos_mode)

svmsmote_ncr=wf.SVMSMOTE_ncr(data_std,30)
print(svmsmote_ncr[30].value_counts())
data_svmsmote_ncr = svmsmote_ncr.drop(columns = [30])
data_svmsmote_ncr=data_svmsmote_ncr.to_numpy()

## SVMSMOTE con OSD (One sided selection)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 30)
data_std = wf.min_max_normalization(datos_mode)

svmsmote_oss=wf.SVMSMOTE_oss(data_std,30)
print(svmsmote_oss[30].value_counts())
data_svmsmote_oss = svmsmote_oss.drop(columns = [30])
data_svmsmote_oss=data_svmsmote_oss.to_numpy()

In [ ]:
y = svmsmote_ncr.iloc[: , 30]
X = svmsmote_ncr.drop(svmsmote_ncr.columns[30], axis=1)
y = y.to_numpy()
X = X.to_numpy()
porcentaje = 0.8
ladoMap = 20

# Inicializa un mapa de colores del tamaño del SOM con todos los valores en blanco
color_map = np.ones((ladoMap, ladoMap, 3))

# Inicializa un mapa de clases del tamaño del SOM con todos los valores en -1
class_map = -1 * np.ones((ladoMap, ladoMap))

# Contadores para cada neurona
counts = np.zeros((ladoMap, ladoMap, 2), dtype=int)

# Itera sobre cada dato y su clase correspondiente
for datum, class_ in zip(X, y):
    # Encuentra la neurona ganadora para este dato
    winner = main_map.calculate_bmu(datum)[1]
    
    # Actualiza el contador de la clase para la neurona ganadora
    counts[int(winner[0]), int(winner[1]), int(class_)] += 1
    
    # Actualiza el mapa de clases con la clase reconocida
    class_map[winner] = class_

# Itera sobre cada neurona para determinar el color final
for i in range(ladoMap):
    for j in range(ladoMap):
        # Calcula el porcentaje de cada clase para la neurona actual
        percent_class_0 = counts[i, j, 0] / (counts[i, j, 0] + counts[i, j, 1]) if counts[i, j, 0] + counts[i, j, 1] > 0 else 0
        percent_class_1 = counts[i, j, 1] / (counts[i, j, 0] + counts[i, j, 1]) if counts[i, j, 0] + counts[i, j, 1] > 0 else 0
        
        # Asigna el color según las condiciones especificadas
        if percent_class_0 >= porcentaje:
            color_map[i, j] = [0, 0, 1]  # Azul
        elif percent_class_1 >= porcentaje:
            color_map[i, j] = [1, 0, 0]  # Rojo
        elif counts[i, j, 0] + counts[i, j, 1] == 0:
            color_map[i, j] = [1, 1, 1]  # Blanco si no ha reconocido ningún patrón
        else:
            color_map[i, j] = [0, 1, 0]  # Verde si no alcanza el % de ninguna clase

# Crea una figura con diferentes tamaños para los subplots
fig = plt.figure(figsize=(8, 6))

# Crea una leyenda para el primer gráfico
legend_elements = [Patch(facecolor='blue', edgecolor='b', label='Clase 0'),
                   Patch(facecolor='red', edgecolor='r', label='Clase 1'),
                   Patch(facecolor='green', edgecolor='g', label='Ambas clases')]
ax1 = fig.add_subplot(121)  # 121 significa "1 fila, 2 columnas, primer gráfico"
#ax1.legend(handles=legend_elements, loc='upper right')
ax1.imshow(np.rot90(color_map))
ax1.set_title('Mapa de neuronas w/o escala')

# Inicializa un mapa de neuronas con ceros
neuron_map = np.zeros((ladoMap, ladoMap))
ganadores = np.zeros((ladoMap, ladoMap))

# Itera sobre los datos y las clases
for datum, clss in zip(X, y):
    # Obtiene la neurona ganadora para el dato actual
    winner = main_map.calculate_bmu(datum)[1]
    ganadores[winner] = 1
    print(winner, clss)
    # Si la clase es 0, resta 1, si la clase es 1, suma 1
    neuron_map[winner[0], winner[1]] += 1 if clss == 1 else -1

# Normaliza el mapa de neuronas para que los valores estén entre -1 y 1
# neuron_map = neuron_map / np.max(np.abs(neuron_map))

# Define los colores de la escala como azul, verde y amarillo
cmap_colors = ["indigo", "royalblue", "limegreen", "yellow"]

# Crea el mapa de colores
cmap = LinearSegmentedColormap.from_list("mycmap", cmap_colors)

# Dibuja el mapa de calor
ax2 = fig.add_subplot(122)  # 122 significa "1 fila, 2 columnas, segundo gráfico"
im = ax2.imshow(np.rot90(neuron_map), cmap=cmap, interpolation='nearest')
ax2.set_title('Mapa de neuronas con escala')

# Muestra la barra de colores
fig.colorbar(im, ax=ax2)

# Ajusta el layout
plt.tight_layout()

# Muestra el gráfico
plt.show()

num_blue = np.sum(np.all(color_map == [0, 0, 1], axis=-1))
num_red = np.sum(np.all(color_map == [1, 0, 0], axis=-1))
num_white = np.sum(np.all(color_map == [1, 1, 1], axis=-1))
num_green = np.sum(np.all(color_map == [0, 1, 0], axis=-1))
# Print the number of neurons of each color
print(f"Number of blue neurons: {num_blue}")
print(f"Number of red neurons: {num_red}")
print(f"Number of white neurons: {num_white}")
print(f"Number of green neurons: {num_green}")

data_balanced_dropped_smote_edited_sint = smote_enn_sinteticos.drop(columns = [3])
data_balanced_dropped_smote_edited_sint=data_balanced_dropped_smote_edited_sint.to_numpy()
main_classification_smoted_enn_sinteticos = GEMA.Classification(main_map, data_balanced_dropped_smote_edited_sint)
GEMA.Visualization.heat_map(main_classification_smoted_enn_sinteticos)

## Clasificación SVM SMOTE GEMA fraudes

In [ ]:
main_classification_svmsmote_tomek = GEMA.Classification(main_map, data_svmsmote_tomek)
print("svmsmote+tommed quantification:",main_classification_svmsmote_tomek.quantization_error)
print("svmsmote+tommed topological:",main_classification_svmsmote_tomek.topological_error)

main_classification_svmsmote_edited= GEMA.Classification(main_map, data_svmsmote_edited)
print("svmsmote+edited quantification:",main_classification_svmsmote_edited.quantization_error)
print("svmsmote+edited topological:",main_classification_svmsmote_edited.topological_error)

main_classification_svmsmote_cnn= GEMA.Classification(main_map, data_svmsmote_cnn)
print("svmsmote+cnn quantification:",main_classification_svmsmote_cnn.quantization_error)
print("svmsmote+cnn topological:",main_classification_svmsmote_cnn.topological_error)

main_classification_svmsmote_ncr= GEMA.Classification(main_map, data_svmsmote_ncr)
print("svmsmote+ncr quantification:",main_classification_svmsmote_ncr.quantization_error)
print("svmsmote+ncr topological:",main_classification_svmsmote_ncr.topological_error)

main_classification_svmsmote_oss= GEMA.Classification(main_map, data_svmsmote_oss)
print("svmsmote+oss quantification:",main_classification_svmsmote_oss.quantization_error)
print("svmsmote+oss topological:",main_classification_svmsmote_oss.topological_error)

## Clasificación Minisom SVM SMOTE fraudes

In [ ]:
print("svmsmote+tommed")
print(som.quantization_error(data_svmsmote_tomek), som.topographic_error(data_svmsmote_tomek))
print("svmsmote+linked")
print(som.quantization_error(data_svmsmote_edited), som.topographic_error(data_svmsmote_edited))
print("svmsmote+cnn")
print(som.quantization_error(data_svmsmote_cnn), som.topographic_error(data_svmsmote_cnn))
print("svmsmote+ncr")
print(som.quantization_error(data_svmsmote_ncr), som.topographic_error(data_svmsmote_ncr))
print("svmsmote+oss")
print(som.quantization_error(data_svmsmote_oss), som.topographic_error(data_svmsmote_oss))

## KMS SMOTE fraudes

In [ ]:
## KMSSMOTE + Tomed_links
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 30)
data_std = wf.min_max_normalization(datos_mode)

kmssmote_tomek=wf.KMSSMOTE_tomek(data_std,30)
print(kmssmote_tomek[30].value_counts())
data_kmssmote_tomek = kmssmote_tomek.drop(columns = [30])
data_kmssmote_tomek=data_kmssmote_tomek.to_numpy()

## KMSSMOTE + ENN
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 30)
data_std = wf.min_max_normalization(datos_mode)

kmssmote_edited=wf.KMSSMOTE_enn(data_std,30)
print(kmssmote_edited[30].value_counts())
data_kmssmote_edited = kmssmote_edited.drop(columns = [30])
data_kmssmote_edited=data_kmssmote_edited.to_numpy()

## KMSSMOTE con CNN (condensed nearest neighbors)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 30)
data_std = wf.min_max_normalization(datos_mode)

kmssmote_cnn=wf.KMSSMOTE_cnn(data_std,30)
print(kmssmote_cnn[30].value_counts())
data_kmssmote_cnn = kmssmote_cnn.drop(columns = [30])
data_kmssmote_cnn=data_kmssmote_cnn.to_numpy()

## KMSSMOTE con NCR (Neighboorhood cleaning rule)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 30)
data_std = wf.min_max_normalization(datos_mode)

kmssmote_ncr=wf.KMSSMOTE_ncr(data_std,30)
print(kmssmote_ncr[30].value_counts())
data_kmssmote_ncr = kmssmote_ncr.drop(columns = [30])
data_kmssmote_ncr=data_kmssmote_ncr.to_numpy()

## KMSSMOTE con OSD (One sided selection)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 30)
data_std = wf.min_max_normalization(datos_mode)

kmssmote_oss=wf.KMSSMOTE_oss(data_std,30)
print(kmssmote_oss[30].value_counts())
data_kmssmote_oss = kmssmote_oss.drop(columns = [30])
data_kmssmote_oss=data_kmssmote_oss.to_numpy()

## Clasificación GEMA KMS SMOTE fraudes

In [ ]:
main_classification_kmssmote_tomek = GEMA.Classification(main_map, data_kmssmote_tomek)
print("svmsmote+tommed quantification:",main_classification_kmssmote_tomek.quantization_error)
print("svmsmote+tommed topological:",main_classification_kmssmote_tomek.topological_error)

main_classification_kmssmote_edited= GEMA.Classification(main_map, data_kmssmote_edited)
print("svmsmote+edited quantification:",main_classification_kmssmote_edited.quantization_error)
print("svmsmote+edited topological:",main_classification_kmssmote_edited.topological_error)

main_classification_kmssmote_cnn= GEMA.Classification(main_map, data_kmssmote_cnn)
print("svmsmote+cnn quantification:",main_classification_kmssmote_cnn.quantization_error)
print("svmsmote+cnn topological:",main_classification_kmssmote_cnn.topological_error)

main_classification_kmssmote_ncr= GEMA.Classification(main_map, data_kmssmote_ncr)
print("svmsmote+ncr quantification:",main_classification_kmssmote_ncr.quantization_error)
print("svmsmote+ncr topological:",main_classification_kmssmote_ncr.topological_error)

main_classification_kmssmote_oss= GEMA.Classification(main_map, data_kmssmote_oss)
print("svmsmote+oss quantification:",main_classification_kmssmote_oss.quantization_error)
print("svmsmote+oss topological:",main_classification_kmssmote_oss.topological_error)

## Clasificación Minisom KMS SMOTE fraudes

In [ ]:
print("kmssmote+tommed")
print(som.quantization_error(data_kmssmote_tomek), som.topographic_error(data_kmssmote_tomek))
print("kmssmote+linked")
print(som.quantization_error(data_kmssmote_edited), som.topographic_error(data_kmssmote_edited))
print("kmssmote+cnn")
print(som.quantization_error(data_kmssmote_cnn), som.topographic_error(data_kmssmote_cnn))
print("kmssmote+ncr")
print(som.quantization_error(data_kmssmote_ncr), som.topographic_error(data_kmssmote_ncr))
print("kmssmote+oss")
print(som.quantization_error(data_kmssmote_oss), som.topographic_error(data_kmssmote_oss))

## Importar dataset 

In [ ]:
## Importar dataset
url = '../../PROYECTO SOM DESBALANCEADO/DATASET/microcalcification.csv'
missing_values = [' ', 'NaN', 'na', 'Na', '-', '--', 'n/a', '?']
columns = ['class']
data = pd.read_csv(url, na_values = missing_values, header=None)
data = data.iloc[1:]
data[6] = data[6].str.strip("'").astype(int)
data[6] = data[6].replace({-1: 0})
data

In [ ]:
data = data.astype(float)
'''# Contar cuántas filas tienen un valor de 0 en la columna "columna_nombre"
filas_con_cero = data[data[30] == 0]

# Verificar si hay suficientes filas con 0 antes de eliminar
if len(filas_con_cero) >= 274000:
    # Eliminar las primeras 180,000 filas con 0
    data = data.drop(filas_con_cero.index[:274000])'''
datos_mode = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(datos_mode, 6)
datos_mode[6].value_counts()

In [75]:
datos_mode=datos_mode.drop(columns = [6])
data_std = wf.min_max_normalization(datos_mode)
target = data[6].values
label_names={0:'Micro', 1:'No micro'}
data_som=data_std.to_numpy()

## Entrenamiento GEMA microcalcificaciones

In [ ]:
main_map = GEMA.Map(data_som, 
                    5, 
                    43000, 
                    initial_lr=0.05275041385159045, 
                    initial_neighbourhood = 0,
                    distance = 'euclidean',
                    use_decay= False,
                    presentation= 'random',
                    weights = 'random')

main_classification = GEMA.Classification(main_map, data_som)
print("quantification error:",main_classification.quantization_error)
print("Topological_error:",main_classification.topological_error)

## Entrenamiento Minisom microcalcificaciones

In [ ]:
n_neurons = 5
m_neurons = 5

som = MiniSom(n_neurons, m_neurons, data_som.shape[1], learning_rate=0.05275041385159045, random_seed=42)
som.train(data_som, num_iteration=43000, verbose=True)  # random training
print("Topological error:",som.topographic_error(data_som))

## SMOTE Micro

In [ ]:
## Smote + Tomed_links
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 6)
data_std = wf.min_max_normalization(datos_mode)

Smote_tomed=wf.smote_tomed_link(data_std,6)
print(Smote_tomed[6].value_counts())
data_balanced_dropped_smote_tomed = Smote_tomed.drop(columns = [6])
data_balanced_dropped_smote_tomed=data_balanced_dropped_smote_tomed.to_numpy()

## smote_edited_nearest_neighbor
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 6)
data_std = wf.min_max_normalization(datos_mode)

Smote_edited=wf.smote_edited_nearest_neighbor(data_std,6)
print(Smote_edited[6].value_counts())
data_balanced_dropped_smote_edited = Smote_edited.drop(columns = [6])
data_balanced_dropped_smote_edited=data_balanced_dropped_smote_edited.to_numpy()

## smote con CNN (condensed nearest neighbors)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 6)
data_std = wf.min_max_normalization(datos_mode)

Smote_cnn=wf.smote_cnn(data_std,6)
print(Smote_cnn[6].value_counts())
data_smote_cnn = Smote_cnn.drop(columns = [6])
data_smote_cnn=data_smote_cnn.to_numpy()

## smote con NCR (Neighboorhood cleaning rule)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 6)
data_std = wf.min_max_normalization(datos_mode)

Smote_ncr=wf.smote_ncr(data_std,6)
print(Smote_ncr[6].value_counts())
data_smote_ncr = Smote_ncr.drop(columns = [6])
data_smote_ncr=data_smote_ncr.to_numpy()

## smote con OSD (One sided selection)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 6)
data_std = wf.min_max_normalization(datos_mode)

Smote_oss=wf.smote_osd(data_std,6)
print(Smote_oss[6].value_counts())
data_smote_oss = Smote_oss.drop(columns = [6])
data_smote_oss=data_smote_oss.to_numpy()

## Clasificación GEMA SMOTE Micro

In [ ]:
main_classification_smoted_tomed = GEMA.Classification(main_map, data_balanced_dropped_smote_tomed)
print("Smote+tommed quantification:",main_classification_smoted_tomed.quantization_error)
print("Smote+tommed topological:",main_classification_smoted_tomed.topological_error)

main_classification_smoted_edited= GEMA.Classification(main_map, data_balanced_dropped_smote_edited)
print("Smote+edited quantification:",main_classification_smoted_edited.quantization_error)
print("Smote+edited topological:",main_classification_smoted_edited.topological_error)

main_classification_smoted_cnn= GEMA.Classification(main_map, data_smote_cnn)
print("Smote+cnn quantification:",main_classification_smoted_cnn.quantization_error)
print("Smote+cnn topological:",main_classification_smoted_cnn.topological_error)

main_classification_smoted_ncr= GEMA.Classification(main_map, data_smote_ncr)
print("Smote+ncr quantification:",main_classification_smoted_ncr.quantization_error)
print("Smote+ncr topological:",main_classification_smoted_ncr.topological_error)

main_classification_smoted_oss= GEMA.Classification(main_map, data_smote_oss)
print("Smote+oss quantification:",main_classification_smoted_oss.quantization_error)
print("Smote+oss topological:",main_classification_smoted_oss.topological_error)

## Clasificación SMOTE Minisom micro

In [ ]:
print("Smote+tommed")
print(som.quantization_error(data_balanced_dropped_smote_tomed), som.topographic_error(data_balanced_dropped_smote_tomed))
print("Smote+linked")
print(som.quantization_error(data_balanced_dropped_smote_edited), som.topographic_error(data_balanced_dropped_smote_edited))
print("Smote+cnn")
print(som.quantization_error(data_smote_cnn), som.topographic_error(data_smote_cnn))
print("Smote+ncr")
print(som.quantization_error(data_smote_ncr), som.topographic_error(data_smote_ncr))
print("Smote+oss")
print(som.quantization_error(data_smote_oss), som.topographic_error(data_smote_oss))

## ADASYN micro

In [ ]:
## ADASYN + Tomed_links
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 6)
data_std = wf.min_max_normalization(datos_mode)

adasyn_tomek=wf.adasyn_tomek(data_std,6)
print(adasyn_tomek[6].value_counts())
data_adasyn_tomek = adasyn_tomek.drop(columns = [6])
data_adasyn_tomek=data_adasyn_tomek.to_numpy()

## ADASYN + ENN
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 6)
data_std = wf.min_max_normalization(datos_mode)

adasyn_edited=wf.adasyn_enn(data_std,6)
print(adasyn_edited[6].value_counts())
data_adasyn_edited = adasyn_edited.drop(columns = [6])
data_adasyn_edited=data_adasyn_edited.to_numpy()

## ADASYN con CNN (condensed nearest neighbors)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 6)
data_std = wf.min_max_normalization(datos_mode)

adasyn_cnn=wf.adasyn_cnn(data_std,6)
print(adasyn_cnn[6].value_counts())
data_adasyn_cnn = adasyn_cnn.drop(columns = [6])
data_adasyn_cnn=data_adasyn_cnn.to_numpy()

## ADASYN con NCR (Neighboorhood cleaning rule)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 6)
data_std = wf.min_max_normalization(datos_mode)

adasyn_ncr=wf.adasyn_ncr(data_std,6)
print(adasyn_ncr[6].value_counts())
data_adasyn_ncr = adasyn_ncr.drop(columns = [6])
data_adasyn_ncr=data_adasyn_ncr.to_numpy()

## ADASYN con OSD (One sided selection)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 6)
data_std = wf.min_max_normalization(datos_mode)

adasyn_oss=wf.adasyn_oss(data_std,6)
print(adasyn_oss[6].value_counts())
data_adasyn_oss = adasyn_oss.drop(columns = [6])
data_adasyn_oss=data_adasyn_oss.to_numpy()

## Clasificacion GEMA ADASYN Micro

In [ ]:
main_classification_adasyn_tomek = GEMA.Classification(main_map, data_adasyn_tomek)
print("adasyn+tommed quantification:",main_classification_adasyn_tomek.quantization_error)
print("adasyn+tommed topological:",main_classification_adasyn_tomek.topological_error)

main_classification_adasyn_edited= GEMA.Classification(main_map, data_adasyn_edited)
print("adasyn+edited quantification:",main_classification_adasyn_edited.quantization_error)
print("adasyn+edited topological:",main_classification_adasyn_edited.topological_error)

main_classification_smoted_cnn= GEMA.Classification(main_map, data_smote_cnn)
print("adasyn+cnn quantification:",main_classification_smoted_cnn.quantization_error)
print("adasyn+cnn topological:",main_classification_smoted_cnn.topological_error)

main_classification_smoted_ncr= GEMA.Classification(main_map, data_adasyn_ncr)
print("adasyn+ncr quantification:",main_classification_smoted_ncr.quantization_error)
print("adasyn+ncr topological:",main_classification_smoted_ncr.topological_error)

main_classification_smoted_oss= GEMA.Classification(main_map, data_adasyn_oss)
print("adasyn+oss quantification:",main_classification_smoted_oss.quantization_error)
print("adasyn+oss topological:",main_classification_smoted_oss.topological_error)

## Clasificación Minisom ADASYN Micro

In [ ]:
print("adasyn+tommed")
print(som.quantization_error(data_adasyn_tomek), som.topographic_error(data_adasyn_tomek))
print("adasyn+linked")
print(som.quantization_error(data_adasyn_edited), som.topographic_error(data_adasyn_edited))
print("adasyn+cnn")
print(som.quantization_error(data_smote_cnn), som.topographic_error(data_smote_cnn))
print("adasyn+ncr")
print(som.quantization_error(data_adasyn_ncr), som.topographic_error(data_adasyn_ncr))
print("adasyn+oss")
print(som.quantization_error(data_adasyn_oss), som.topographic_error(data_adasyn_oss))

## BSMOTE Micro

In [ ]:
## BSMOTE + Tomed_links
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 6)
data_std = wf.min_max_normalization(datos_mode)

bsmote_tomek=wf.BSMOTE_tomek(data_std,6)
print(bsmote_tomek[6].value_counts())
data_bsmote_tomek = bsmote_tomek.drop(columns = [6])
data_bsmote_tomek=data_bsmote_tomek.to_numpy()

## BSMOTE + ENN
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 6)
data_std = wf.min_max_normalization(datos_mode)

bsmote_edited=wf.BSMOTE_enn(data_std,6)
print(bsmote_edited[6].value_counts())
data_bsmote_edited = bsmote_edited.drop(columns = [6])
data_bsmote_edited=data_bsmote_edited.to_numpy()

## BSMOTE con CNN (condensed nearest neighbors)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 6)
data_std = wf.min_max_normalization(datos_mode)

bsmote_cnn=wf.BSMOTE_cnn(data_std,6)
print(bsmote_cnn[6].value_counts())
data_bsmote_cnn = bsmote_cnn.drop(columns = [6])
data_bsmote_cnn=data_bsmote_cnn.to_numpy()

## BSMOTE con NCR (Neighboorhood cleaning rule)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 6)
data_std = wf.min_max_normalization(datos_mode)

bsmote_ncr=wf.BSMOTE_ncr(data_std,6)
print(bsmote_ncr[6].value_counts())
data_bsmote_ncr = bsmote_ncr.drop(columns = [6])
data_bsmote_ncr=data_bsmote_ncr.to_numpy()

## BSMOTE con OSD (One sided selection)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 6)
data_std = wf.min_max_normalization(datos_mode)

bsmote_oss=wf.BSMOTE_oss(data_std,6)
print(bsmote_oss[6].value_counts())
data_bsmote_oss = bsmote_oss.drop(columns = [6])
data_bsmote_oss=data_bsmote_oss.to_numpy()

## Clasificación GEMA BSMOTE Micro

In [ ]:
main_classification_bsmote_tomek = GEMA.Classification(main_map, data_bsmote_tomek)
print("bsmote+tommed quantification:",main_classification_bsmote_tomek.quantization_error)
print("bsmote+tommed topological:",main_classification_bsmote_tomek.topological_error)

main_classification_bsmote_edited= GEMA.Classification(main_map, data_bsmote_edited)
print("bsmote+edited quantification:",main_classification_bsmote_edited.quantization_error)
print("bsmote+edited topological:",main_classification_bsmote_edited.topological_error)

main_classification_bsmote_cnn= GEMA.Classification(main_map, data_bsmote_cnn)
print("bsmote+cnn quantification:",main_classification_bsmote_cnn.quantization_error)
print("bsmote+cnn topological:",main_classification_bsmote_cnn.topological_error)

main_classification_bsmote_ncr= GEMA.Classification(main_map, data_bsmote_ncr)
print("bsmote+ncr quantification:",main_classification_bsmote_ncr.quantization_error)
print("bsmote+ncr topological:",main_classification_bsmote_ncr.topological_error)

main_classification_bsmote_oss= GEMA.Classification(main_map, data_bsmote_oss)
print("bsmote+oss quantification:",main_classification_bsmote_oss.quantization_error)
print("bsmote+oss topological:",main_classification_bsmote_oss.topological_error)

## Clasificación Minisom BSMOTE Micro

In [ ]:
print("bsmote+tommed")
print(som.quantization_error(data_bsmote_tomek), som.topographic_error(data_bsmote_tomek))
print("bsmote+linked")
print(som.quantization_error(data_bsmote_edited), som.topographic_error(data_bsmote_edited))
print("bsmote+cnn")
print(som.quantization_error(data_bsmote_cnn), som.topographic_error(data_bsmote_cnn))
print("bsmote+ncr")
print(som.quantization_error(data_bsmote_ncr), som.topographic_error(data_bsmote_ncr))
print("bsmote+oss")
print(som.quantization_error(data_bsmote_oss), som.topographic_error(data_bsmote_oss))

## SVM SMOTE Micro

In [ ]:
## SVMSMOTE + Tomed_links
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 6)
data_std = wf.min_max_normalization(datos_mode)

svmsmote_tomek=wf.SVMSMOTE_tomek(data_std,6)
print(svmsmote_tomek[6].value_counts())
data_svmsmote_tomek = svmsmote_tomek.drop(columns = [6])
data_svmsmote_tomek=data_svmsmote_tomek.to_numpy()

## SVMSMOTE + ENN
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 6)
data_std = wf.min_max_normalization(datos_mode)

svmsmote_edited=wf.SVMSMOTE_enn(data_std,6)
print(svmsmote_edited[6].value_counts())
data_svmsmote_edited = svmsmote_edited.drop(columns = [6])
data_svmsmote_edited=data_svmsmote_edited.to_numpy()

## SVMSMOTE con CNN (condensed nearest neighbors)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 6)
data_std = wf.min_max_normalization(datos_mode)

svmsmote_cnn=wf.SVMSMOTE_cnn(data_std,6)
print(svmsmote_cnn[6].value_counts())
data_svmsmote_cnn = svmsmote_cnn.drop(columns = [6])
data_svmsmote_cnn=data_svmsmote_cnn.to_numpy()

## SVMSMOTE con NCR (Neighboorhood cleaning rule)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 6)
data_std = wf.min_max_normalization(datos_mode)

svmsmote_ncr=wf.SVMSMOTE_ncr(data_std,6)
print(svmsmote_ncr[6].value_counts())
data_svmsmote_ncr = svmsmote_ncr.drop(columns = [6])
data_svmsmote_ncr=data_svmsmote_ncr.to_numpy()

## SVMSMOTE con OSD (One sided selection)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 6)
data_std = wf.min_max_normalization(datos_mode)

svmsmote_oss=wf.SVMSMOTE_oss(data_std,6)
print(svmsmote_oss[6].value_counts())
data_svmsmote_oss = svmsmote_oss.drop(columns = [6])
data_svmsmote_oss=data_svmsmote_oss.to_numpy()

## Clasificación GEMA SVM SMOTE Micro

In [ ]:
main_classification_svmsmote_tomek = GEMA.Classification(main_map, data_svmsmote_tomek)
print("svmsmote+tommed quantification:",main_classification_svmsmote_tomek.quantization_error)
print("svmsmote+tommed topological:",main_classification_svmsmote_tomek.topological_error)

main_classification_svmsmote_edited= GEMA.Classification(main_map, data_svmsmote_edited)
print("svmsmote+edited quantification:",main_classification_svmsmote_edited.quantization_error)
print("svmsmote+edited topological:",main_classification_svmsmote_edited.topological_error)

main_classification_svmsmote_cnn= GEMA.Classification(main_map, data_svmsmote_cnn)
print("svmsmote+cnn quantification:",main_classification_svmsmote_cnn.quantization_error)
print("svmsmote+cnn topological:",main_classification_svmsmote_cnn.topological_error)

main_classification_svmsmote_ncr= GEMA.Classification(main_map, data_svmsmote_ncr)
print("svmsmote+ncr quantification:",main_classification_svmsmote_ncr.quantization_error)
print("svmsmote+ncr topological:",main_classification_svmsmote_ncr.topological_error)

main_classification_svmsmote_oss= GEMA.Classification(main_map, data_svmsmote_oss)
print("svmsmote+oss quantification:",main_classification_svmsmote_oss.quantization_error)
print("svmsmote+oss topological:",main_classification_svmsmote_oss.topological_error)

## Clasificación SVM SMOTE Minisom Micro

In [ ]:
print("svmsmote+tommed")
print(som.quantization_error(data_svmsmote_tomek), som.topographic_error(data_svmsmote_tomek))
print("svmsmote+linked")
print(som.quantization_error(data_svmsmote_edited), som.topographic_error(data_svmsmote_edited))
print("svmsmote+cnn")
print(som.quantization_error(data_svmsmote_cnn), som.topographic_error(data_svmsmote_cnn))
print("svmsmote+ncr")
print(som.quantization_error(data_svmsmote_ncr), som.topographic_error(data_svmsmote_ncr))
print("svmsmote+oss")
print(som.quantization_error(data_svmsmote_oss), som.topographic_error(data_svmsmote_oss))

## KMS SMOTE Micro

In [ ]:
## KMSSMOTE + Tomed_links
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 6)
data_std = wf.min_max_normalization(datos_mode)

kmssmote_tomek=wf.KMSSMOTE_tomek(data_std,6)
print(kmssmote_tomek[6].value_counts())
data_kmssmote_tomek = kmssmote_tomek.drop(columns = [6])
data_kmssmote_tomek=data_kmssmote_tomek.to_numpy()

## KMSSMOTE + ENN
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 6)
data_std = wf.min_max_normalization(datos_mode)

kmssmote_edited=wf.KMSSMOTE_enn(data_std,6)
print(kmssmote_edited[6].value_counts())
data_kmssmote_edited = kmssmote_edited.drop(columns = [6])
data_kmssmote_edited=data_kmssmote_edited.to_numpy()

## KMSSMOTE con CNN (condensed nearest neighbors)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 6)
data_std = wf.min_max_normalization(datos_mode)

kmssmote_cnn=wf.KMSSMOTE_cnn(data_std,6)
print(kmssmote_cnn[6].value_counts())
data_kmssmote_cnn = kmssmote_cnn.drop(columns = [6])
data_kmssmote_cnn=data_kmssmote_cnn.to_numpy()

## KMSSMOTE con NCR (Neighboorhood cleaning rule)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 6)
data_std = wf.min_max_normalization(datos_mode)

kmssmote_ncr=wf.KMSSMOTE_ncr(data_std,6)
print(kmssmote_ncr[6].value_counts())
data_kmssmote_ncr = kmssmote_ncr.drop(columns = [6])
data_kmssmote_ncr=data_kmssmote_ncr.to_numpy()

## KMSSMOTE con OSD (One sided selection)
data_dropped = wf.drop_missing_values_columns(data, 5)
datos_mode = wf.mode_imputation(data_dropped, 6)
data_std = wf.min_max_normalization(datos_mode)

kmssmote_oss=wf.KMSSMOTE_oss(data_std,6)
print(kmssmote_oss[6].value_counts())
data_kmssmote_oss = kmssmote_oss.drop(columns = [6])
data_kmssmote_oss=data_kmssmote_oss.to_numpy()

## Clasificación GEMA KMS SMOTE Micro

In [ ]:
main_classification_kmssmote_tomek = GEMA.Classification(main_map, data_kmssmote_tomek)
print("svmsmote+tommed quantification:",main_classification_kmssmote_tomek.quantization_error)
print("svmsmote+tommed topological:",main_classification_kmssmote_tomek.topological_error)

main_classification_kmssmote_edited= GEMA.Classification(main_map, data_kmssmote_edited)
print("svmsmote+edited quantification:",main_classification_kmssmote_edited.quantization_error)
print("svmsmote+edited topological:",main_classification_kmssmote_edited.topological_error)

main_classification_kmssmote_cnn= GEMA.Classification(main_map, data_kmssmote_cnn)
print("svmsmote+cnn quantification:",main_classification_kmssmote_cnn.quantization_error)
print("svmsmote+cnn topological:",main_classification_kmssmote_cnn.topological_error)

main_classification_kmssmote_ncr= GEMA.Classification(main_map, data_kmssmote_ncr)
print("svmsmote+ncr quantification:",main_classification_kmssmote_ncr.quantization_error)
print("svmsmote+ncr topological:",main_classification_kmssmote_ncr.topological_error)

main_classification_kmssmote_oss= GEMA.Classification(main_map, data_kmssmote_oss)
print("svmsmote+oss quantification:",main_classification_kmssmote_oss.quantization_error)
print("svmsmote+oss topological:",main_classification_kmssmote_oss.topological_error)

## Clasificación KMS SMOTE Minisom Micro

In [ ]:
print("kmssmote+tommed")
print(som.quantization_error(data_kmssmote_tomek), som.topographic_error(data_kmssmote_tomek))
print("kmssmote+linked")
print(som.quantization_error(data_kmssmote_edited), som.topographic_error(data_kmssmote_edited))
print("kmssmote+cnn")
print(som.quantization_error(data_kmssmote_cnn), som.topographic_error(data_kmssmote_cnn))
print("kmssmote+ncr")
print(som.quantization_error(data_kmssmote_ncr), som.topographic_error(data_kmssmote_ncr))
print("kmssmote+oss")
print(som.quantization_error(data_kmssmote_oss), som.topographic_error(data_kmssmote_oss))